In [ ]:
!pip install coloredlogs
!pip install --upgrade --force-reinstall tensorflow==1.15.0 
!pip install ujson

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)
#drive.flush_and_unmount()
!ls "/content/gdrive/My Drive/test-data-Eye-tracking/GazeML/"
##!cp "/content/gdrive/My Drive/test-data-Eye-tracking/MPIIGaze.h5" "MPIIGaze.h5"
#!ls "/content/gdrive/My Drive/test-data-Eye-tracking/imgs/imgs"
#dir = "/content/gdrive/My Drive/test-data-Eye-tracking/GazeML/test-elg"
!ls "/content/gdrive/My Drive/test-data-Eye-tracking/GazeML/test-elg"


#!rm -rf "/content/gdrive/My Drive/test-data-Eye-tracking/GazeML-elg-outputs/"*

In [ ]:
!unzip "/content/gdrive/My Drive/test-data-Eye-tracking/imgs.zip" -d "/content/data/"


In [ ]:
"""##download weights and training data
!wget -Nnv https://raw.githubusercontent.com/aveenakottwani/EyeGazeEstimationModels/main/Gaze%20Estimations/trainedModel/ELG_i180x108_f60x36_n64_m3.zip
!wget -Nnv https://raw.githubusercontent.com/aveenakottwani/EyeGazeEstimationModels/main/Gaze%20Estimations/trainedModel/test7.mp4
!rm -rf "/content/outputs/ELG_i180x108_f60x36_n64_m3/"
!ls "/content/outputs/ELG_i180x108_f60x36_n64_m3/"

"""

In [ ]:
!wget -Nnv "https://raw.githubusercontent.com/aveenakottwani/EyeGazeEstimationModels/main/Gaze%20Estimations/trainedModel/ELG_i60x36_f60x36_n32_m2.zip"

In [ ]:
!unzip "/content/ELG_i60x36_f60x36_n32_m2.zip" -d "/content/outputs/"


In [ ]:
image_dir = "/content/data/imgs/" #"/content/gdrive/My Drive/test-data-Eye-tracking/imgs/imgs" #/content/gdrive/My Drive/test-data-Eye-tracking/data/imgs"


In [ ]:
dir = "/content/outputs/"
!ls "/content/outputs/ELG_i60x36_f60x36_n32_m2/checkpoints"
!ls "/content/gdrive/My Drive/test-data-Eye-tracking/data/"

In [ ]:
##datasource

"""Default specification of a data source."""
from collections import OrderedDict
import multiprocessing
import queue
import threading
import time

import numpy as np
import tensorflow as tf

import logging
logger = logging.getLogger(__name__)


class BaseDataSource(object):
    """Base DataSource class."""

    def __init__(self,
                 tensorflow_session: tf.Session,
                 data_format: str = 'NHWC',
                 batch_size: int = 32,
                 num_threads: int = max(4, multiprocessing.cpu_count()),
                 min_after_dequeue: int = 1000,
                 fread_queue_capacity: int = 0,
                 preprocess_queue_capacity: int = 0,
                 staging=False,
                 shuffle=None,
                 testing=False,
                 ):
        """Initialize a data source instance."""
        assert tensorflow_session is not None and isinstance(tensorflow_session, tf.Session)
        assert isinstance(batch_size, int) and batch_size > 0
        if shuffle is None:
            shuffle = staging
        self.testing = testing
        if testing:
            assert not shuffle and not staging
            # if num_threads != 1:
            #     logger.info('Forcing use of single thread for live testing.')
            # num_threads = 1
        self.staging = staging
        self.shuffle = shuffle
        self.data_format = data_format.upper()
        assert self.data_format == 'NHWC' or self.data_format == 'NCHW'
        self.batch_size = batch_size
        self.num_threads = num_threads
        self._tensorflow_session = tensorflow_session
        self._coordinator = tf.train.Coordinator()
        self.all_threads = []

        # Setup file read queue
        self._fread_queue_capacity = fread_queue_capacity
        if self._fread_queue_capacity == 0:
            self._fread_queue_capacity = (num_threads + 1) * batch_size
        self._fread_queue = queue.Queue(maxsize=self._fread_queue_capacity)

        with tf.variable_scope(''.join(c for c in self.short_name if c.isalnum())):
            # Setup preprocess queue
            labels, dtypes, shapes = self._determine_dtypes_and_shapes()
            self._preprocess_queue_capacity = (min_after_dequeue + (num_threads + 1) * batch_size
                                               if preprocess_queue_capacity == 0
                                               else preprocess_queue_capacity)
            if shuffle:
                self._preprocess_queue = tf.RandomShuffleQueue(
                        capacity=self._preprocess_queue_capacity,
                        min_after_dequeue=min_after_dequeue,
                        dtypes=dtypes, shapes=shapes,
                )
            else:
                self._preprocess_queue = tf.FIFOQueue(
                        capacity=self._preprocess_queue_capacity,
                        dtypes=dtypes, shapes=shapes,
                )
            self._tensors_to_enqueue = OrderedDict([
                (label, tf.placeholder(dtype, shape=shape, name=label))
                for label, dtype, shape in zip(labels, dtypes, shapes)
            ])

            self._enqueue_op = \
                self._preprocess_queue.enqueue(tuple(self._tensors_to_enqueue.values()))
            self._preprocess_queue_close_op = \
                self._preprocess_queue.close(cancel_pending_enqueues=True)
            self._preprocess_queue_size_op = self._preprocess_queue.size()
            self._preprocess_queue_clear_op = \
                self._preprocess_queue.dequeue_up_to(self._preprocess_queue.size())
            if not staging:
                output_tensors = self._preprocess_queue.dequeue_many(self.batch_size)
                if not isinstance(output_tensors, list):
                    output_tensors = [output_tensors]
                self._output_tensors = dict([
                    (label, tensor) for label, tensor in zip(labels, output_tensors)
                ])
            else:
                # Setup on-GPU staging area
                self._staging_area = tf.contrib.staging.StagingArea(
                    dtypes=dtypes,
                    shapes=[tuple([batch_size] + list(shape)) for shape in shapes],
                    capacity=1,  # This does not have to be high
                )
                self._staging_area_put_op = \
                    self._staging_area.put(self._preprocess_queue.dequeue_many(batch_size))
                self._staging_area_clear_op = self._staging_area.clear()

                self._output_tensors = dict([
                    (label, tensor) for label, tensor in zip(labels, self._staging_area.get())
                ])

        logger.info('Initialized data source: "%s"' % self.short_name)

    def __del__(self):
        """Destruct and clean up instance."""
        self.cleanup()

    @property
    def num_entries(self):
        """Number of entries in this data source.
        Used to calculate number of steps to train when asked to be trained for # epochs.
        """
        raise NotImplementedError('BaseDataSource::num_entries not specified.')

    @property
    def short_name(self):
        """Short identifier for data source.
        Overload this magic method if the class is generic, eg. supporting h5py/numpy arrays as
        input with specific data sources.
        """
        raise NotImplementedError('BaseDataSource::short_name not specified.')

    __cleaned_up = False

    def cleanup(self):
        """Force-close all threads."""
        if self.__cleaned_up:
            return

        # Clear queues
        fread_threads = [t for t in self.all_threads if t.name.startswith('fread_')]
        preprocess_threads = [t for t in self.all_threads if t.name.startswith('preprocess_')]
        transfer_threads = [t for t in self.all_threads if t.name.startswith('transfer_')]

        self._coordinator.request_stop()

        # Unblock any self._fread_queue.put calls
        while True:
            try:
                self._fread_queue.get_nowait()
            except queue.Empty:
                break
            time.sleep(0.1)

        # Push data through to trigger exits in preprocess/transfer threads
        for _ in range(self.batch_size * self.num_threads):
            self._fread_queue.put(None)
        self._tensorflow_session.run(self._preprocess_queue_close_op)
        if self.staging:
            self._tensorflow_session.run(self._staging_area_clear_op)

        self._coordinator.join(self.all_threads, stop_grace_period_secs=5)
        self.__cleaned_up = True

    def reset(self):
        """Reset threads and empty queues (where possible)."""
        assert self.testing is True

        # Clear queues
        self._coordinator.request_stop()
        with self._fread_queue.mutex:  # Unblock any self._fread_queue.get calls
            self._fread_queue.queue.clear()
        for _ in range(2*self.num_threads):
            self._fread_queue.put(None)
        while True:  # Unblock any enqueue requests
            preprocess_queue_size = self._tensorflow_session.run(self._preprocess_queue_size_op)
            if preprocess_queue_size == 0:
                break
            self._tensorflow_session.run(self._preprocess_queue_clear_op)
            time.sleep(0.1)
        while True:  # Unblock any self._fread_queue.put calls
            try:
                self._fread_queue.get_nowait()
            except queue.Empty:
                break
            time.sleep(0.1)
        self._coordinator.join(self.all_threads, stop_grace_period_secs=5)

        # Restart threads
        self._coordinator.clear_stop()
        self.create_and_start_threads()

    def _determine_dtypes_and_shapes(self):
        """Determine the dtypes and shapes of Tensorflow queue and staging area entries."""
        while True:
            raw_entry = next(self.entry_generator(yield_just_one=True))
            if raw_entry is None:
                continue
            preprocessed_entry_dict = self.preprocess_entry(raw_entry)
            if preprocessed_entry_dict is not None:
                break
        labels, values = zip(*list(preprocessed_entry_dict.items()))
        dtypes = [value.dtype for value in values]
        shapes = [value.shape for value in values]
        return labels, dtypes, shapes

    def entry_generator(self, yield_just_one=False):
        """Return a generator which reads an entry from disk or memory.
        This method should be thread-safe so make sure to use threading.Lock where necessary.
        The implemented method should explicitly handle the `yield_just_one=True` case to only
        yield one entry without hanging in the middle of an infinite loop.
        """
        raise NotImplementedError('BaseDataSource::entry_generator not implemented.')

    def preprocess_entry(self, entry):
        """Preprocess a "raw" data entry and yield a dict.
        Each element of an entry is provided to this method as separate arguments.
        This method should be thread-safe so make sure to use threading.Lock where necessary.
        """
        raise NotImplementedError('BaseDataSource::preprocess_entry not implemented.')

    def read_entry_job(self):
        """Job to read an entry and enqueue to _fread_queue."""
        read_entry = self.entry_generator()
        while not self._coordinator.should_stop():
            try:
                entry = next(read_entry)
            except StopIteration:
                if not self.testing:
                    continue
                else:
                    logger.debug('Reached EOF in %s' % threading.current_thread().name)
                    break
            if entry is not None:
                self._fread_queue.put(entry)
        read_entry.close()
        logger.debug('Exiting thread %s' % threading.current_thread().name)

    def preprocess_job(self):
        """Job to fetch and preprocess an entry."""
        while not self._coordinator.should_stop():
            raw_entry = self._fread_queue.get()
            if raw_entry is None:
                return
            preprocessed_entry_dict = self.preprocess_entry(raw_entry)
            if preprocessed_entry_dict is not None:
                feed_dict = dict([(self._tensors_to_enqueue[label], value)
                                  for label, value in preprocessed_entry_dict.items()])
                try:
                    self._tensorflow_session.run(self._enqueue_op, feed_dict=feed_dict)
                except (tf.errors.CancelledError, RuntimeError):
                    break
        logger.debug('Exiting thread %s' % threading.current_thread().name)

    def transfer_to_gpu_job(self):
        """Transfer a data entry from CPU memory to GPU memory."""
        while not self._coordinator.should_stop():
            try:
                self._tensorflow_session.run(self._staging_area_put_op)
            except tf.errors.CancelledError or tf.errors.OutOfRangeError:
                break
        logger.debug('Exiting thread %s' % threading.current_thread().name)

    def create_threads(self):
        """Create Python threads for multi-threaded read and preprocess jobs."""
        name = self.short_name
        self.all_threads = []

        def _create_and_register_thread(*args, **kwargs):
            thread = threading.Thread(*args, **kwargs)
            thread.daemon = True
            self.all_threads.append(thread)

        for i in range(self.num_threads):
            # File read thread
            _create_and_register_thread(target=self.read_entry_job, name='fread_%s_%d' % (name, i))

            # Preprocess thread
            _create_and_register_thread(target=self.preprocess_job,
                                        name='preprocess_%s_%d' % (name, i))

        if self.staging:
            # Send-to-GPU thread
            _create_and_register_thread(target=self.transfer_to_gpu_job,
                                        name='transfer_%s_%d' % (name, i))

    def start_threads(self):
        """Begin executing all created threads."""
        assert len(self.all_threads) > 0
        for thread in self.all_threads:
            thread.start()

    def create_and_start_threads(self):
        """Create and begin threads for preprocessing."""
        self.create_threads()
        self.start_threads()

    @property
    def output_tensors(self):
        """Return tensors holding a preprocessed batch."""
        return self._output_tensors

In [ ]:
##CheckpointManager


"""Manage saving and loading of model checkpoints."""
import os
import re

import numpy as np
import tensorflow as tf

import logging
logger = logging.getLogger(__name__)


class CheckpointManager(object):
    """Manager to coordinate saving and loading of trainable parameters."""

    def __init__(self, model):
        """Initialize manager based on given model instance."""
        self._tensorflow_session = model._tensorflow_session
        self._model = model

    def build_savers(self):
        """Create tf.train.Saver instances."""
        all_saveable_vars = sorted(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES) +
                                   tf.get_collection(tf.GraphKeys.SAVEABLE_OBJECTS) +
                                   tf.get_collection(tf.GraphKeys.MOVING_AVERAGE_VARIABLES) +
                                   tf.get_collection_ref('batch_norm_non_trainable'),
                                   key=lambda v: v.name)

        # Grab all available prefixes
        all_prefixes = []
        for v in all_saveable_vars:
            name = v.name
            if '/' not in name:
                continue
            prefix = name.split('/')[0]
            if prefix == 'test' or prefix == 'learning_params':
                continue
            if prefix not in all_prefixes:
                all_prefixes.append(prefix)

        # For each prefix, create saver
        self._savers = {}
        for prefix in all_prefixes:
            vars_to_save = [v for v in all_saveable_vars if v.name.startswith(prefix + '/')]
            if len(vars_to_save):
                self._savers[prefix] = tf.train.Saver(vars_to_save, max_to_keep=2)

    def load_all(self):
        """Load all available weights for each known prefix."""
        print("in load_all - to restore")
        iteration_number = 0
        iteration_numbers = []
        for prefix, saver in self._savers.items():
            output_path = '%s/checkpoints/%s' % (self._model.output_path, prefix)
            checkpoint = tf.train.get_checkpoint_state(output_path)
            print("load_all - output path where restore is tried from",self._model.output_path,prefix,checkpoint)
            if checkpoint and checkpoint.model_checkpoint_path:
                checkpoint_name = os.path.basename(checkpoint.model_checkpoint_path)
                try:  # Attempt to restore saveable variables
                    print("loadd_all - output path where restore is tried from",output_path,checkpoint_name,prefix)
                    self._savers[prefix].restore(self._tensorflow_session,
                                                 '%s/%s' % (output_path, checkpoint_name))
                    print("in load_all - tried restore")
                    iteration_numbers.append(
                        int(next(re.finditer("(\d+)(?!.*\d)", checkpoint_name)).group(0))
                    )
                except Exception as e:
                    import traceback
                    traceback.print_exc()
        if len(iteration_numbers) > 0:
            iteration_number = np.amax(iteration_numbers)
        return iteration_number

    def save_all(self, iteration_number):
        """Save all prefixes."""
        prefixes_to_use = []
        for schedule in self._model._learning_schedule:
            for prefixes in schedule['loss_terms_to_optimize'].values():
                prefixes_to_use += prefixes
        prefixes_to_use = list(set(prefixes_to_use))

        for prefix, saver in self._savers.items():
            if prefix not in prefixes_to_use:
                continue
            output_path = '%s/checkpoints/%s' % (self._model.output_path, prefix)
            if not os.path.isdir(output_path):
                os.makedirs(output_path)
            saver.save(self._tensorflow_session, output_path + '/model',
                       global_step=iteration_number)
            logger.debug('Saved %s' % output_path)
        logger.info('CheckpointManager::save_all call done')

In [ ]:
##SummaryManager

"""Manage registration and evaluation of summary operations."""
import numpy as np
import tensorflow as tf

import logging
logger = logging.getLogger(__name__)


class SummaryManager(object):
    """Manager to remember and run summary operations as necessary."""

    def __init__(self, model, cheap_ops_every_n_secs=2, expensive_ops_every_n_mins=2):
        """Initialize manager based on given model instance."""
        self._tensorflow_session = model._tensorflow_session
        self._model = model
        self._cheap_ops = {
            'train': {},
            'test': {},
            'full_test': {},
        }
        self._expensive_ops = {
            'train': {},
            'test': {},
            'full_test': {},
        }
        self._cheap_ops_every_n_secs = cheap_ops_every_n_secs
        self._expensive_ops_every_n_secs = 60 * expensive_ops_every_n_mins

        self._ready_to_write = False

    def _prepare_for_write(self):
        """Merge together cheap and expensive ops separately."""
        self._writer = tf.summary.FileWriter(self._model.output_path,
                                             self._tensorflow_session.graph)
        for mode in ('train', 'test', 'full_test'):
            self._expensive_ops[mode].update(self._cheap_ops[mode])
        self._ready_to_write = True

    def get_ops(self, mode='train'):
        """Retrieve summary ops to evaluate at given iteration number."""
        if not self._ready_to_write:
            self._prepare_for_write()
        if mode == 'test' or mode == 'full_test':  # Always return all ops for test case
            return self._expensive_ops[mode]
        elif mode == 'train':  # Select ops to evaluate based on defined frequency
            check_func = self._model.time.has_been_n_seconds_since_last
            if check_func('expensive_summaries_train', self._expensive_ops_every_n_secs):
                return self._expensive_ops[mode]
            elif check_func('cheap_summaries_train', self._cheap_ops_every_n_secs):
                return self._cheap_ops[mode]
        return {}

    def write_summaries(self, summary_outputs, iteration_number):
        """Write given outputs to `self._writer`."""
        for _, summary in summary_outputs.items():
            self._writer.add_summary(summary, global_step=iteration_number)

    def _get_clean_name(self, operation):
        name = operation.name

        # Determine mode
        mode = 'train'
        if name.startswith('test/') or name.startswith('test_data/'):
            mode = 'test'
        elif name.startswith('loss/test/') or name.startswith('metric/test/'):
            mode = 'full_test'

        # Correct name
        if mode == 'test':
            name = name[name.index('/') + 1:]
        elif mode == 'full_test':
            name = '/'.join(name.split('/')[2:])
        if name[-2] == ':':
            name = name[:-2]
        return mode, name

    def _register_cheap_op(self, operation):
        mode, name = self._get_clean_name(operation)
        try:
            assert name not in self._cheap_ops[mode] and name not in self._expensive_ops[mode]
        except AssertionError:
            raise Exception('Duplicate definition of summary item: "%s"' % name)
        self._cheap_ops[mode][name] = operation

    def _register_expensive_op(self, operation):
        mode, name = self._get_clean_name(operation)
        try:
            assert name not in self._cheap_ops[mode] and name not in self._expensive_ops[mode]
        except AssertionError:
            raise Exception('Duplicate definition of summary item: "%s"' % name)
        self._expensive_ops[mode][name] = operation

    def audio(self, name, tensor, **kwargs):
        """TODO: Log summary of audio."""
        raise NotImplementedError('SummaryManager::audio not implemented.')

    def text(self, name, tensor, **kwargs):
        """TODO: Log summary of text."""
        raise NotImplementedError('SummaryManager::text not implemented.')

    def histogram(self, name, tensor, **kwargs):
        """TODO: Log summary of audio."""
        operation = tf.summary.histogram(name, tensor, **kwargs)
        self._register_expensive_op(operation)

    def image(self, name, tensor, data_format='channels_last', **kwargs):
        """TODO: Log summary of image."""
        if data_format == 'channels_first':
            tensor = tf.transpose(tensor, perm=(0, 2, 3, 1))
        c = tensor.shape.as_list()[-1]
        if c == 3:  # Assume RGB and convert to BGR for visualization
            tensor = tensor[:, :, :, ::-1]   # TODO: find better solution
        operation = tf.summary.image(name, tensor, **kwargs)
        self._register_expensive_op(operation)

    def _4d_tensor(self, name, tensor, **kwargs):
        """Display all filters in a grid for visualization."""
        h, w, c, num_tensor = tensor.shape.as_list()

        # Try to visualise convolutional filters or feature maps
        # See: https://gist.github.com/kukuruza/03731dc494603ceab0c5
        # input shape: (Y, X, C, N)
        if c != 1 and c != 3:
            tensor = tf.reduce_mean(tensor, axis=2, keep_dims=True)
            c = 1
        # shape is now: (Y, X, 1|C, N)
        v_min = tf.reduce_min(tensor)
        v_max = tf.reduce_max(tensor)
        tensor -= v_min
        tensor *= 1.0 / (v_max - v_min)
        tensor = tf.pad(tensor, [[1, 0], [1, 0], [0, 0], [0, 0]], 'CONSTANT')
        tensor = tf.transpose(tensor, perm=(3, 0, 1, 2))
        # shape is now: (N, Y, X, C)
        # place tensor on grid
        num_tensor_x = int(np.round(np.sqrt(num_tensor)))
        num_tensor_y = num_tensor / num_tensor_x
        while not num_tensor_y.is_integer():
            num_tensor_x += 1
            num_tensor_y = num_tensor / num_tensor_x
        num_tensor_y = int(num_tensor_y)
        h += 1
        w += 1
        tensor = tf.reshape(tensor, (num_tensor_x, h * num_tensor_y, w, c))
        # shape is now: (N_x, Y * N_y, X, c)
        tensor = tf.transpose(tensor, (0, 2, 1, 3))
        # shape is now: (N_x, X, Y * N_y, c)
        tensor = tf.reshape(tensor, (1, w * num_tensor_x, h * num_tensor_y, c))
        # shape is now: (1, X * N_x, Y * N_y, c)
        tensor = tf.transpose(tensor, (0, 2, 1, 3))
        # shape is now: (1, Y * N_y, X * N_x, c)
        tensor = tf.pad(tensor, [[0, 0], [0, 1], [0, 1], [0, 0]], 'CONSTANT')

        self.image(name, tensor, **kwargs)

    def filters(self, name, tensor, **kwargs):
        """Log summary of convolutional filters.
        Note that this method expects the output of the convolutional layer when using
        `tf.layers.conv2d` or for the filters to be defined in the same scope as the output tensor.
        """
        assert 'data_format' not in kwargs
        with tf.name_scope('viz_filters'):
            # Find tensor holding trainable kernel weights
            name_stem = '/'.join(tensor.name.split('/')[:-1]) + '/kernel'
            matching_tensors = [t for t in tf.trainable_variables() if t.name.startswith(name_stem)]
            assert len(matching_tensors) == 1
            filters = matching_tensors[0]

            # H x W x C x N
            h, w, c, n = filters.shape.as_list()
            filters = tf.transpose(filters, perm=(3, 2, 0, 1))
            # N x C x H x W
            filters = tf.reshape(filters, (n*c, 1, h, w))
            # NC x 1 x H x W
            filters = tf.transpose(filters, perm=(2, 3, 1, 0))
            # H x W x 1 x NC

            self._4d_tensor(name, filters, **kwargs)

    def feature_maps(self, name, tensor, mean_across_channels=True, data_format='channels_last',
                     **kwargs):
        """Log summary of feature maps / image activations."""
        with tf.name_scope('viz_featuremaps'):
            if data_format == 'channels_first':
                # N x C x H x W
                tensor = tf.transpose(tensor, perm=(0, 2, 3, 1))
            # N x H x W x C
            if mean_across_channels:
                tensor = tf.reduce_mean(tensor, axis=3, keepdims=True)
                # N x H x W x 1
                tensor = tf.transpose(tensor, perm=(1, 2, 3, 0))
            else:
                n, c, h, w = tensor.shape.as_list()
                tensor = tf.reshape(tensor, (n*c, 1, h, w))
                # N x 1 x H x W
                tensor = tf.transpose(tensor, perm=(2, 3, 1, 0))
            # H x W x 1 x N

            self._4d_tensor(name, tensor, **kwargs)

    def tiled_images(self, name, tensor, data_format='channels_last', **kwargs):
        """Log summary of feature maps / image activations."""
        with tf.name_scope('viz_featuremaps'):
            if data_format == 'channels_first':
                # N x C x H x W
                tensor = tf.transpose(tensor, perm=(0, 2, 3, 1))
            # N x H x W x C
            tensor = tf.transpose(tensor, perm=(1, 2, 3, 0))
            # H x W x C x N
            self._4d_tensor(name, tensor, **kwargs)

    def scalar(self, name, tensor, **kwargs):
        """Log summary of scalar."""
        operation = tf.summary.scalar(name, tensor, **kwargs)
        self._register_cheap_op(operation)

In [ ]:
##TimeManager

"""Routines to time events and restrict logs or operations by frequency."""
import time

import numpy as np

import logging
logger = logging.getLogger(__name__)


class TimeManager(object):
    """Manage timing of event executions or measure timings."""

    def __init__(self, model):
        """Initialize manager based on given model instance."""
        self._tensorflow_session = model._tensorflow_session
        self._model = model

        self._timers = {}
        self._last_time = {}

    def start(self, name, **kwargs):
        """Begin timer for given event/operation."""
        if name not in self._timers:
            timer = Timer(**kwargs)
            self._timers[name] = timer
        else:
            timer = self._timers[name]
        timer.start()

    def end(self, name):
        """End timer for given event/operation."""
        assert name in self._timers
        return self._timers[name].end()

    def has_been_n_seconds_since_last(self, identifier, seconds):
        """Indicate if enough time has passed since last time.
        Also updates the `last time` record based on identifier.
        """
        current_time = time.time()
        if identifier not in self._last_time or \
           (current_time - self._last_time[identifier] > seconds):
            self._last_time[identifier] = current_time
            return True
        return False

    def log_every(self, identifier, message, seconds=1):
        """Limit logging of messages based on specified interval and identifier."""
        if self.has_been_n_seconds_since_last(identifier, seconds):
            logger.info(message)
        else:
            logger.debug(message)


# A local Timer class for timing
class Timer(object):
    """Record start and end times as requested and provide summaries."""

    def __init__(self, average_over_last_n_timings=10):
        """Store keyword parameters."""
        self._average_over_last_n_timings = average_over_last_n_timings
        self._active = False
        self._timings = []
        self._start_time = -1

    def start(self):
        """Cache starting time."""
        # assert not self._active
        self._start_time = time.time()
        self._active = True

    def end(self):
        """Check ending time and store difference."""
        assert self._active and self._start_time > 0

        # Calculate difference
        end_time = time.time()
        time_difference = end_time - self._start_time

        # Record timing (and trim history)
        self._timings.append(time_difference)
        if len(self._timings) > self._average_over_last_n_timings:
            self._timings = self._timings[-self._average_over_last_n_timings:]

        # Reset
        self._start_time = -1
        self._active = False

        return time_difference

    @property
    def current_mean(self):
        """Calculate mean timing for as many trials as specified in constructor."""
        values = self._timings
        return np.mean(values)

In [ ]:
##LiveTester

"""Concurrent testing during training."""
import collections
import platform
import threading
import time
import traceback

import numpy as np
import tensorflow as tf

import logging
logger = logging.getLogger(__name__)


class LiveTester(object):
    """Manage concurrent testing on test data source."""

    def __init__(self, model, data_source, use_batch_statistics=True):
        """Initialize tester with reference to model and data sources."""
        self.model = model
        self.data = data_source
        self.time = self.model.time
        self.summary = self.model.summary
        self._tensorflow_session = model._tensorflow_session

        self._is_testing = False
        self._condition = threading.Condition()

        self._use_batch_statistics = use_batch_statistics

    def stop(self):
        logger.info('LiveTester::stop is being called.')
        self._is_testing = False

    def __del__(self):
        """Handle deletion of instance by closing thread."""
        if not hasattr(self, '_coordinator'):
            return
        self._coordinator.request_stop()
        with self._condition:
            self._is_testing = True  # Break wait if waiting
            self._condition.notify_all()
        self._coordinator.join([self._thread], stop_grace_period_secs=1)

    def _true_if_testing(self):
        return self._is_testing

    def trigger_test_if_not_testing(self, current_step):
        """If not currently testing, run test."""
        if not self._is_testing:
            with self._condition:
                self._is_testing = True
                self._testing_at_step = current_step
                self._condition.notify_all()

    def test_job(self):
        """Evaluate requested metric over entire test set."""
        while not self._coordinator.should_stop():
            with self._condition:
                self._condition.wait_for(self._true_if_testing)
                if self._coordinator.should_stop():
                    break
                should_stop = False
                try:
                    should_stop = self.do_full_test()
                except:
                    traceback.print_exc()
                self._is_testing = False
                if should_stop is True:
                    break
        logger.debug('Exiting thread %s' % threading.current_thread().name)

    def do_full_test(self, sleep_between_batches=0.2):
        # Copy current weights over
        self.copy_model_weights()

        # Reset data sources
        for data_source_name, data_source in self.data.items():
            data_source.reset()
            num_batches = int(data_source.num_entries / data_source.batch_size)

        # Decide what to evaluate
        fetches = self._tensors_to_evaluate
        outputs = dict([(name, list()) for name in fetches.keys()])

        # Select random index to produce (image) summaries at
        summary_index = np.random.randint(num_batches)

        self.time.start('full test')
        for i in range(num_batches):
            if self._is_testing is not True:
                logger.debug('Testing flag found to be `False` at iter. %d' % i)
                break
            logger.debug('Testing on %03d/%03d batches.' % (i + 1, num_batches))
            if i == summary_index:
                fetches['summaries'] = self.summary.get_ops(mode='test')
            try:
                output = self._tensorflow_session.run(
                    fetches=fetches,
                    feed_dict={
                        self.model.is_training: False,
                        self.model.use_batch_statistics: self._use_batch_statistics,
                    },
                )
            except (tf.errors.CancelledError, RuntimeError):
                return True
            time.sleep(sleep_between_batches)  # Brief pause to prioritise training
            if 'summaries' in output:  # Write summaries on first batch
                self.summary.write_summaries(output['summaries'], self._testing_at_step)
                del fetches['summaries']
                del output['summaries']
            for name, value in output.items():  # Gather results from this batch
                outputs[name].append(output[name])
        self.time.end('full test')

        # If incomplete, skip this round of tests (most likely shutting down)
        if len(list(outputs.values())[0]) != num_batches:
            return True

        # Calculate mean values
        for name, values in outputs.items():
            outputs[name] = np.mean(values)

        # TODO: Log metric as summary
        to_print = '[Test at step %06d] ' % self._testing_at_step
        to_print += ', '.join([
            '%s = %f' % (name, value) for name, value in outputs.items()
        ])
        logger.info(to_print)

        # Store mean metrics/losses (and other summaries)
        feed_dict = dict([(self._placeholders[name], value)
                         for name, value in outputs.items()])
        feed_dict[self.model.is_training] = False
        feed_dict[self.model.use_batch_statistics] = True
        try:
            summaries = self._tensorflow_session.run(
                fetches=self.summary.get_ops(mode='full_test'),
                feed_dict=feed_dict,
            )
        except (tf.errors.CancelledError, RuntimeError):
            return True
        self.summary.write_summaries(summaries, self._testing_at_step)

        return False

    def do_final_full_test(self, current_step):
        logger.info('Stopping the live testing threads.')

        # Stop thread(s)
        self._is_testing = False
        self._coordinator.request_stop()
        with self._condition:
            self._is_testing = True  # Break wait if waiting
            self._condition.notify_all()
        self._coordinator.join([self._thread], stop_grace_period_secs=1)

        # Start final full test
        logger.info('Running final full test')
        self.copy_model_weights()
        self._is_testing = True
        self._testing_at_step = current_step
        self.do_full_test(sleep_between_batches=0)

    def _post_model_build(self):
        """Prepare combined operation to copy model parameters over from CPU/GPU to CPU."""
        with tf.variable_scope('copy2test'):
            all_variables = tf.global_variables()
            train_vars = dict([(v.name, v) for v in all_variables
                               if not v.name.startswith('test/')])
            test_vars = dict([(v.name, v) for v in all_variables
                              if v.name.startswith('test/')])
            self._copy_variables_to_test_model_op = tf.tuple([
                test_vars['test/' + k].assign(train_vars[k]) for k in train_vars.keys()
                if 'test/' + k in test_vars
            ])

        # Begin testing thread
        self._coordinator = tf.train.Coordinator()
        self._thread = threading.Thread(target=self.test_job,
                                        name='%s_tester' % self.model.identifier)
        self._thread.daemon = True
        self._thread.start()

        # Pick tensors we need to evaluate
        all_tensors = dict(self.model.loss_terms['test'], **self.model.metrics['test'])
        self._tensors_to_evaluate = dict([(n, t) for n, t in all_tensors.items()])
        loss_terms_to_evaluate = dict([(n, t) for n, t in self.model.loss_terms['test'].items()
                                       if t in self._tensors_to_evaluate.values()])
        metrics_to_evaluate = dict([(n, t) for n, t in self.model.metrics['test'].items()
                                    if t in self._tensors_to_evaluate.values()])

        # Placeholders for writing summaries at end of test run
        self._placeholders = {}
        for type_, tensors in (('loss', loss_terms_to_evaluate),
                               ('metric', metrics_to_evaluate)):
            for name in tensors.keys():
                name = '%s/test/%s' % (type_, name)
                placeholder = tf.placeholder(dtype=np.float32, name=name + '_placeholder')
                self.summary.scalar(name, placeholder)
                self._placeholders[name.split('/')[-1]] = placeholder

    def copy_model_weights(self):
        """Copy weights from main model used for training.
        This operation should stop-the-world, that is, training should not occur.
        """
        assert self._copy_variables_to_test_model_op is not None
        self._tensorflow_session.run(self._copy_variables_to_test_model_op)
##Model        logger.debug('Copied over trainable model parameters for testing.')

In [ ]:


"""Base model class for Tensorflow-based model construction."""

import os
import sys
import time
from typing import Any, Dict, List

import numpy as np
import tensorflow as tf
"""from .data_source import BaseDataSource
from .live_tester import LiveTester
from .time_manager import TimeManager
from .summary_manager import SummaryManager
from .checkpoint_manager import CheckpointManager"""
import logging
logger = logging.getLogger(__name__)


class BaseModel(object):
    """Base model class for Tensorflow-based model construction.
    This class assumes that there exist no other Tensorflow models defined.
    That is, any variable that exists in the Python session will be grabbed by the class.
    """

    def __init__(self,
                 tensorflow_session: tf.Session,
                 learning_schedule: List[Dict[str, Any]] = [],
                 train_data: Dict[str, BaseDataSource] = {},
                 test_data: Dict[str, BaseDataSource] = {},
                 test_losses_or_metrics: str = None,
                 use_batch_statistics_at_test: bool = True,
                 identifier: str = None):
        """Initialize model with data sources and parameters."""
        self._tensorflow_session = tensorflow_session
        self._train_data = train_data
        self._test_data = test_data
        self._test_losses_or_metrics = test_losses_or_metrics
        self._initialized = False
        self.__identifier = identifier

        # Extract and keep known prefixes/scopes
        self._learning_schedule = learning_schedule
        self._known_prefixes = [schedule for schedule in learning_schedule]

        # Check consistency of given data sources
        train_data_sources = list(train_data.values())
        test_data_sources = list(test_data.values())
        all_data_sources = train_data_sources + test_data_sources
        first_data_source = all_data_sources.pop()
        self._batch_size = first_data_source.batch_size
        self._data_format = first_data_source.data_format
        for data_source in all_data_sources:
            if data_source.batch_size != self._batch_size:
                raise ValueError(('Data source "%s" has anomalous batch size of %d ' +
                                  'when detected batch size is %d.') % (data_source.short_name,
                                                                        data_source.batch_size,
                                                                        self._batch_size))
            if data_source.data_format != self._data_format:
                raise ValueError(('Data source "%s" has anomalous data_format of %s ' +
                                  'when detected data_format is %s.') % (data_source.short_name,
                                                                         data_source.data_format,
                                                                         self._data_format))
        self._data_format_longer = ('channels_first' if self._data_format == 'NCHW'
                                    else 'channels_last')

        # Make output dir
        if not os.path.isdir(self.output_path):
            os.makedirs(self.output_path)
        #self.output_path = dir

        # Log messages to file
        root_logger = logging.getLogger()
        file_handler = logging.FileHandler(self.output_path + '/messages.log')
        file_handler.setFormatter(root_logger.handlers[0].formatter)
        for handler in root_logger.handlers[1:]:  # all except stdout
            root_logger.removeHandler(handler)
        root_logger.addHandler(file_handler)

        # Register a manager for tf.Summary
        self.summary = SummaryManager(self)

        # Register a manager for checkpoints
        self.checkpoint = CheckpointManager(self)

        # Register a manager for timing related operations
        self.time = TimeManager(self)

        # Prepare for live (concurrent) validation/testing during training, on the CPU
        self._enable_live_testing = (len(self._train_data) > 0) and (len(self._test_data) > 0)
        self._tester = LiveTester(self, self._test_data, use_batch_statistics_at_test)

        # Run-time parameters
        with tf.variable_scope('learning_params'):
            self.is_training = tf.placeholder(tf.bool)
            self.use_batch_statistics = tf.placeholder(tf.bool)
            self.learning_rate_multiplier = tf.Variable(1.0, trainable=False, dtype=tf.float32)
            self.learning_rate_multiplier_placeholder = tf.placeholder(dtype=tf.float32)
            self.assign_learning_rate_multiplier = \
                tf.assign(self.learning_rate_multiplier, self.learning_rate_multiplier_placeholder)

        self._build_all_models()

    def __del__(self):
        """Explicitly call methods to cleanup any live threads."""
        train_data_sources = list(self._train_data.values())
        test_data_sources = list(self._test_data.values())
        all_data_sources = train_data_sources + test_data_sources
        for data_source in all_data_sources:
            data_source.cleanup()
        self._tester.__del__()

    __identifier_stem = None

    @property
    def identifier(self):
        """Identifier for model based on time."""
        if self.__identifier is not None:  # If loading from checkpoints or having naming enforced
            return self.__identifier
        if self.__identifier_stem is None:
            self.__identifier_stem = self.__class__.__name__ + '/' + time.strftime('%y%m%d%H%M%S')
        return self.__identifier_stem + self._identifier_suffix

    @property
    def _identifier_suffix(self):
        """Identifier suffix for model based on data sources and parameters."""
        return ''

    @property
    def output_path(self):
        """Path to store logs and model weights into."""
        """return '%s/%s' % (os.path.abspath(os.path.dirname(__file__) + '/../../outputs'),
                          self.identifier)"""
        return '%s/%s' % (dir,
                          self.identifier)

    def _build_all_models(self):
        """Build training (GPU/CPU) and testing (CPU) streams."""
        self.output_tensors = {}
        self.loss_terms = {}
        self.metrics = {}

        def _build_datasource_summaries(data_sources, mode):
            """Register summary operations for input data from given data sources."""
            with tf.variable_scope('%s_data' % mode):
                for data_source_name, data_source in data_sources.items():
                    tensors = data_source.output_tensors
                    for key, tensor in tensors.items():
                        summary_name = '%s/%s' % (data_source_name, key)
                        shape = tensor.shape.as_list()
                        num_dims = len(shape)
                        if num_dims == 4:  # Image data
                            if shape[1] == 1 or shape[1] == 3:
                                self.summary.image(summary_name, tensor,
                                                   data_format='channels_first')
                            elif shape[3] == 1 or shape[3] == 3:
                                self.summary.image(summary_name, tensor,
                                                   data_format='channels_last')
                            # TODO: fix issue with no summary otherwise
                        elif num_dims == 2:
                            self.summary.histogram(summary_name, tensor)
                        else:
                            logger.debug('I do not know how to create a summary for %s (%s)' %
                                         (summary_name, tensor.shape.as_list()))

        def _build_train_or_test(mode):
            data_sources = self._train_data if mode == 'train' else self._test_data

            # Build model
            output_tensors, loss_terms, metrics = self.build_model(data_sources, mode=mode)

            # Record important tensors
            self.output_tensors[mode] = output_tensors
            self.loss_terms[mode] = loss_terms
            self.metrics[mode] = metrics

            # Create summaries for scalars
            if mode == 'train':
                for name, loss_term in loss_terms.items():
                    self.summary.scalar('loss/%s/%s' % (mode, name), loss_term)
                for name, metric in metrics.items():
                    self.summary.scalar('metric/%s/%s' % (mode, name), metric)

        # Build the main model
        if len(self._train_data) > 0:
            _build_datasource_summaries(self._train_data, mode='train')
            _build_train_or_test(mode='train')
            logger.info('Built model.')

            # Print no. of parameters and lops
            flops = tf.profiler.profile(
                options=tf.profiler.ProfileOptionBuilder(
                    tf.profiler.ProfileOptionBuilder.float_operation()
                ).with_empty_output().build())
            logger.info('------------------------------')
            logger.info(' Approximate Model Statistics ')
            logger.info('------------------------------')
            logger.info('FLOPS per input: {:,}'.format(flops.total_float_ops / self._batch_size))
            logger.info(
                'Trainable Parameters: {:,}'.format(
                    np.sum([np.prod(v.shape.as_list()) for v in tf.trainable_variables()])
                )
            )
            logger.info('------------------------------')

        # If there are any test data streams, build same model with different scope
        # Trainable parameters will be copied at test time
        if len(self._test_data) > 0:
            _build_datasource_summaries(self._test_data, mode='test')
            with tf.variable_scope('test'):
                _build_train_or_test(mode='test')
            logger.info('Built model for live testing.')

        if self._enable_live_testing:
            self._tester._post_model_build()  # Create copy ops to be run before every test run

    def build_model(self, data_sources: Dict[str, BaseDataSource], mode: str):
        """Build model."""
        raise NotImplementedError('BaseModel::build_model is not yet implemented.')

    def initialize_if_not(self, training=False):
        """Initialize variables and begin preprocessing threads."""
        if self._initialized:
            return

        # Build supporting operations
        with tf.variable_scope('savers'):
            self.checkpoint.build_savers()  # Create savers
        if training:
            with tf.variable_scope('optimize'):
                self._build_optimizers()

        # Start pre-processing routines
        for _, datasource in self._train_data.items():
            datasource.create_and_start_threads()

        # Initialize all variables
        self._tensorflow_session.run(tf.global_variables_initializer())
        self._initialized = True

    def _build_optimizers(self):
        """Based on learning schedule, create optimizer instances."""
        self._optimize_ops = []
        all_trainable_variables = tf.trainable_variables()
        all_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        all_reg_losses = tf.losses.get_regularization_losses()
        for spec in self._learning_schedule:
            optimize_ops = []
            update_ops = []
            loss_terms = spec['loss_terms_to_optimize']
            reg_losses = []
            assert isinstance(loss_terms, dict)
            for loss_term_key, prefixes in loss_terms.items():
                assert loss_term_key in self.loss_terms['train'].keys()
                variables_to_train = []
                for prefix in prefixes:
                    variables_to_train += [
                        v for v in all_trainable_variables
                        if v.name.startswith(prefix)
                    ]
                    update_ops += [
                        o for o in all_update_ops
                        if o.name.startswith(prefix)
                    ]
                    reg_losses += [
                        l for l in all_reg_losses
                        if l.name.startswith(prefix)
                    ]

                optimizer_class = tf.train.AdamOptimizer
                optimizer = optimizer_class(
                    learning_rate=self.learning_rate_multiplier * spec['learning_rate'],
                    # beta1=0.9,
                    # beta2=0.999,
                )
                final_loss = self.loss_terms['train'][loss_term_key]
                if len(reg_losses) > 0:
                    final_loss += tf.reduce_sum(reg_losses)
                with tf.control_dependencies(update_ops):
                    gradients, variables = zip(*optimizer.compute_gradients(
                        loss=final_loss,
                        var_list=variables_to_train,
                        aggregation_method=tf.AggregationMethod.EXPERIMENTAL_ACCUMULATE_N,
                    ))
                    # gradients, _ = tf.clip_by_global_norm(gradients, 5.0)  # TODO: generalize
                    optimize_op = optimizer.apply_gradients(zip(gradients, variables))
                optimize_ops.append(optimize_op)
            self._optimize_ops.append(optimize_ops)
            logger.info('Built optimizer for: %s' % ', '.join(loss_terms.keys()))

    def train_loop_pre(self, current_step):
        """Run this at beginning of training loop."""
        pass

    def train_loop_post(self, current_step):
        """Run this at end of training loop."""
        pass

    def train(self, num_epochs=None, num_steps=None):
        """Train model as requested."""
        if num_steps is None:
            num_entries = np.min([s.num_entries for s in list(self._train_data.values())])
            print("num_entries",num_entries)
            num_steps = int(num_epochs * num_entries / self._batch_size)
            print("steps",num_steps)
        self.initialize_if_not(training=True)

        try:
            initial_step = self.checkpoint.load_all()
            current_step = initial_step
            for current_step in range(initial_step, num_steps):
                print("current_step",current_step)
                # Extra operations defined in implementation of this base class
                self.train_loop_pre(current_step)

                # Select loss terms, optimize operations, and metrics tensors to evaluate
                fetches = {}
                schedule_id = current_step % len(self._learning_schedule)
                schedule = self._learning_schedule[schedule_id]
                fetches['optimize_ops'] = self._optimize_ops[schedule_id]
                loss_term_keys, _ = zip(*list(schedule['loss_terms_to_optimize'].items()))
                fetches['loss_terms'] = [self.loss_terms['train'][k] for k in loss_term_keys]
                summary_op = self.summary.get_ops(mode='train')
                if len(summary_op) > 0:
                    fetches['summaries'] = summary_op

                # Run one optimization iteration and retrieve calculated loss values
                self.time.start('train_iteration', average_over_last_n_timings=100)
                outcome = self._tensorflow_session.run(
                    fetches=fetches,
                    feed_dict={
                        self.is_training: True,
                        self.use_batch_statistics: True,
                    }
                )
                self.time.end('train_iteration')

                # Print progress
                to_print = '%07d> ' % current_step
                to_print += ', '.join(['%s = %g' % (k, v)
                                       for k, v in zip(loss_term_keys, outcome['loss_terms'])])
                self.time.log_every('train_iteration', to_print, seconds=2)

                # Trigger copy weights & concurrent testing (if not already running)
                if self._enable_live_testing:
                    self._tester.trigger_test_if_not_testing(current_step)

                # Write summaries
                if 'summaries' in outcome:
                    self.summary.write_summaries(outcome['summaries'], current_step)

                # Save model weights
                if self.time.has_been_n_seconds_since_last('save_weights', 600) \
                        and current_step > initial_step:
                    self.checkpoint.save_all(current_step)

                # Extra operations defined in implementation of this base class
                self.train_loop_post(current_step)

        except KeyboardInterrupt:
            # Handle CTRL-C graciously
            self.checkpoint.save_all(current_step)
            sys.exit(0)

        # Stop live testing, and run final full test
        if self._enable_live_testing:
            self._tester.do_final_full_test(current_step)

        # Save final weights
        if current_step > initial_step:
            self.checkpoint.save_all(current_step)

    def inference_generator(self):
        """Perform inference on test data and yield a batch of output."""
        self.initialize_if_not(training=False)
        self.checkpoint.load_all()  # Load available weights

        # TODO: Make more generic by not picking first source
        print("self._train_data.values()",len(self._train_data.values()))
        data_source = next(iter(self._train_data.values()))
        while True:
            fetches = dict(self.output_tensors['train'], **data_source.output_tensors)
            start_time = time.time()
            outputs = self._tensorflow_session.run(
                fetches=fetches,
                feed_dict={
                    self.is_training: False,
                    self.use_batch_statistics: True,
                },
            )
            print("generate one output")
            outputs['inference_time'] = 1e3*(time.time() - start_time)
            yield outputs
        

In [ ]:
##frames

"""Data source of stream of frames."""
import bz2
import dlib
import queue
import shutil
import threading
import time
from typing import Tuple
import os
from urllib.request import urlopen

import cv2 as cv
import numpy as np
import tensorflow as tf

##from core import BaseDataSource


class FramesSource(BaseDataSource):
    """Preprocessing of stream of frames."""

    def __init__(self,
                 tensorflow_session: tf.Session,
                 batch_size: int,
                 eye_image_shape: Tuple[int, int],
                 staging: bool=False,
                 **kwargs):
        """Create queues and threads to read and preprocess data."""
        self._eye_image_shape = eye_image_shape
        self._proc_mutex = threading.Lock()
        self._read_mutex = threading.Lock()

        self._frame_read_queue = queue.Queue(maxsize=1)
        self._frame_read_thread = threading.Thread(target=self.frame_read_job, name='frame_read')
        self._frame_read_thread.daemon = True
        self._frame_read_thread.start()

        self._current_index = 0
        self._last_frame_index = 0
        self._indices = []
        self._frames = {}
        self._open = True

        # Call parent class constructor
        super().__init__(tensorflow_session, batch_size=batch_size, num_threads=1,
                         fread_queue_capacity=batch_size, preprocess_queue_capacity=batch_size,
                         shuffle=False, staging=staging, **kwargs)

    _short_name = 'Frames'

    @property
    def short_name(self):
        """Short name specifying source."""
        return self._short_name

    def frame_read_job(self):
        """Read frame from webcam."""
        generate_frame = self.frame_generator()
        while True:
            before_frame_read = time.time()
            bgr = next(generate_frame)
            if bgr is not None:
                after_frame_read = time.time()
                with self._read_mutex:
                    self._frame_read_queue.queue.clear()
                    self._frame_read_queue.put_nowait((before_frame_read, bgr, after_frame_read))
        self._open = False

    def frame_generator(self):
        """Read frame from webcam."""
        raise NotImplementedError('Frames::frame_generator not implemented.')

    def entry_generator(self, yield_just_one=False):
        """Generate eye image entries by detecting faces and facial landmarks."""
        try:
            while range(1) if yield_just_one else True:
                # Grab frame
                with self._proc_mutex:
                    before_frame_read, bgr, after_frame_read = self._frame_read_queue.get()
                    bgr = cv.flip(bgr, flipCode=1)  # Mirror
                    current_index = self._last_frame_index + 1
                    self._last_frame_index = current_index

                    grey = cv.cvtColor(bgr, cv.COLOR_BGR2GRAY)
                    frame = {
                        'frame_index': current_index,
                        'time': {
                            'before_frame_read': before_frame_read,
                            'after_frame_read': after_frame_read,
                        },
                        'bgr': bgr,
                        'grey': grey,
                    }
                    self._frames[current_index] = frame
                    self._indices.append(current_index)

                    # Keep just a few frames around
                    frames_to_keep = 120
                    if len(self._indices) > frames_to_keep:
                        for index in self._indices[:-frames_to_keep]:
                            del self._frames[index]
                        self._indices = self._indices[-frames_to_keep:]

                # Eye image segmentation pipeline
                self.detect_faces(frame)
                self.detect_landmarks(frame)
                self.calculate_smoothed_landmarks(frame)
                self.segment_eyes(frame)
                self.update_face_boxes(frame)
                frame['time']['after_preprocessing'] = time.time()

                for i, eye_dict in enumerate(frame['eyes']):
                    yield {
                        'frame_index': np.int64(current_index),
                        'eye': eye_dict['image'],
                        'eye_index': np.uint8(i),
                    }

        finally:
            # Execute any cleanup operations as necessary
            pass

    def preprocess_entry(self, entry):
        """Preprocess segmented eye images for use as neural network input."""
        eye = entry['eye']
        eye = cv.equalizeHist(eye)
        eye = eye.astype(np.float32)
        eye *= 2.0 / 255.0
        eye -= 1.0
        eye = np.expand_dims(eye, -1 if self.data_format == 'NHWC' else 0)
        entry['eye'] = eye
        return entry

    def detect_faces(self, frame):
        """Detect all faces in a frame."""
        frame_index = frame['frame_index']
        previous_index = self._indices[self._indices.index(frame_index) - 1]
        previous_frame = self._frames[previous_index]
        if ('last_face_detect_index' not in previous_frame or
                frame['frame_index'] - previous_frame['last_face_detect_index'] > 59):
            detector = get_face_detector()
            if detector.__class__.__name__ == 'CascadeClassifier':
                detections = detector.detectMultiScale(frame['grey'])
            else:
                detections = detector(cv.resize(frame['grey'], (0, 0), fx=0.5, fy=0.5), 0)
            faces = []
            for d in detections:
                try:
                    l, t, r, b = d.rect.left(), d.rect.top(), d.rect.right(), d.rect.bottom()
                    l *= 2
                    t *= 2
                    r *= 2
                    b *= 2
                    w, h = r - l, b - t
                except AttributeError:  # Using OpenCV LBP detector on CPU
                    l, t, w, h = d
                faces.append((l, t, w, h))
            faces.sort(key=lambda bbox: bbox[0])
            frame['faces'] = faces
            frame['last_face_detect_index'] = frame['frame_index']

            # Clear previous known landmarks. This is to disable smoothing when new face detect
            # occurs. This allows for recovery of drifted detections.
            previous_frame['landmarks'] = []
        else:
            frame['faces'] = previous_frame['faces']
            frame['last_face_detect_index'] = previous_frame['last_face_detect_index']

    def detect_landmarks(self, frame):
        """Detect 5-point facial landmarks for faces in frame."""
        predictor = get_landmarks_predictor()
        landmarks = []
        for face in frame['faces']:
            l, t, w, h = face
            rectangle = dlib.rectangle(left=int(l), top=int(t), right=int(l+w), bottom=int(t+h))
            landmarks_dlib = predictor(frame['grey'], rectangle)

            def tuple_from_dlib_shape(index):
                p = landmarks_dlib.part(index)
                return (p.x, p.y)

            num_landmarks = landmarks_dlib.num_parts
            landmarks.append(np.array([tuple_from_dlib_shape(i) for i in range(num_landmarks)]))
        frame['landmarks'] = landmarks

    _smoothing_window_size = 10
    _smoothing_coefficient_decay = 0.5
    _smoothing_coefficients = None

    def calculate_smoothed_landmarks(self, frame):
        """If there are previous landmark detections, try to smooth current prediction."""
        # Cache coefficients based on defined sliding window size
        if self._smoothing_coefficients is None:
            coefficients = np.power(self._smoothing_coefficient_decay,
                                    list(reversed(list(range(self._smoothing_window_size)))))
            coefficients /= np.sum(coefficients)
            self._smoothing_coefficients = coefficients.reshape(-1, 1)

        # Get a window of frames
        current_index = self._indices.index(frame['frame_index'])
        a = current_index - self._smoothing_window_size + 1
        if a < 0:
            """If slice extends before last known frame."""
            return
        window_indices = self._indices[a:current_index + 1]
        window_frames = [self._frames[idx] for idx in window_indices]
        window_num_landmark_entries = np.array([len(f['landmarks']) for f in window_frames])
        if np.any(window_num_landmark_entries == 0):
            """Any frame has zero faces detected."""
            return
        if not np.all(window_num_landmark_entries == window_num_landmark_entries[0]):
            """Not the same number of faces detected in entire window."""
            return

        # Apply coefficients to landmarks in window
        window_landmarks = np.asarray([f['landmarks'] for f in window_frames])
        frame['smoothed_landmarks'] = np.sum(
            np.multiply(window_landmarks.reshape(self._smoothing_window_size, -1),
                        self._smoothing_coefficients),
            axis=0,
        ).reshape(window_num_landmark_entries[-1], -1, 2)

    def segment_eyes(self, frame):
        """From found landmarks in previous steps, segment eye image."""
        eyes = []

        # Final output dimensions
        oh, ow = self._eye_image_shape

        # Select which landmarks (raw/smoothed) to use
        frame_landmarks = (frame['smoothed_landmarks'] if 'smoothed_landmarks' in frame
                           else frame['landmarks'])

        for face, landmarks in zip(frame['faces'], frame_landmarks):
            # Segment eyes
            # for corner1, corner2, is_left in [(36, 39, True), (42, 45, False)]:
            for corner1, corner2, is_left in [(2, 3, True), (0, 1, False)]:
                x1, y1 = landmarks[corner1, :]
                x2, y2 = landmarks[corner2, :]
                eye_width = 1.5 * np.linalg.norm(landmarks[corner1, :] - landmarks[corner2, :])
                if eye_width == 0.0:
                    continue
                cx, cy = 0.5 * (x1 + x2), 0.5 * (y1 + y2)

                # Centre image on middle of eye
                translate_mat = np.asmatrix(np.eye(3))
                translate_mat[:2, 2] = [[-cx], [-cy]]
                inv_translate_mat = np.asmatrix(np.eye(3))
                inv_translate_mat[:2, 2] = -translate_mat[:2, 2]

                # Rotate to be upright
                roll = 0.0 if x1 == x2 else np.arctan((y2 - y1) / (x2 - x1))
                rotate_mat = np.asmatrix(np.eye(3))
                cos = np.cos(-roll)
                sin = np.sin(-roll)
                rotate_mat[0, 0] = cos
                rotate_mat[0, 1] = -sin
                rotate_mat[1, 0] = sin
                rotate_mat[1, 1] = cos
                inv_rotate_mat = rotate_mat.T

                # Scale
                scale = ow / eye_width
                scale_mat = np.asmatrix(np.eye(3))
                scale_mat[0, 0] = scale_mat[1, 1] = scale
                inv_scale = 1.0 / scale
                inv_scale_mat = np.asmatrix(np.eye(3))
                inv_scale_mat[0, 0] = inv_scale_mat[1, 1] = inv_scale

                # Centre image
                centre_mat = np.asmatrix(np.eye(3))
                centre_mat[:2, 2] = [[0.5 * ow], [0.5 * oh]]
                inv_centre_mat = np.asmatrix(np.eye(3))
                inv_centre_mat[:2, 2] = -centre_mat[:2, 2]

                # Get rotated and scaled, and segmented image
                transform_mat = centre_mat * scale_mat * rotate_mat * translate_mat
                inv_transform_mat = (inv_translate_mat * inv_rotate_mat * inv_scale_mat *
                                     inv_centre_mat)
                eye_image = cv.warpAffine(frame['grey'], transform_mat[:2, :], (ow, oh))
                if is_left:
                    eye_image = np.fliplr(eye_image)
                eyes.append({
                    'image': eye_image,
                    'inv_landmarks_transform_mat': inv_transform_mat,
                    'side': 'left' if is_left else 'right',
                })
        frame['eyes'] = eyes

    def update_face_boxes(self, frame):
        """Update face bounding box based on detected landmarks."""
        frame_landmarks = (frame['smoothed_landmarks'] if 'smoothed_landmarks' in frame
                           else frame['landmarks'])
        for i, (face, landmarks) in enumerate(zip(frame['faces'], frame_landmarks)):
            x_min, y_min = np.amin(landmarks, axis=0)
            x_max, y_max = np.amax(landmarks, axis=0)
            x_mid, y_mid = 0.5 * (x_max + x_min), 0.5 * (y_max + y_min)
            w, h = x_max - x_min, y_max - y_min
            new_w = 2.2 * max(h, w)
            half_w = 0.5 * new_w
            frame['faces'][i] = (int(x_mid - half_w), int(y_mid - half_w), int(new_w), int(new_w))

            # x1, y1 = landmarks[0, :]
            # x2, y2 = landmarks[3, :]
            # face_width = 2.5 * np.sqrt((x1 - x2)**2 + (y1 - y2)**2)
            # if face_width == 0.0:
            #     continue
            #
            # cx, cy = landmarks[4, :]
            # roll = 0.0 if x1 == x2 else np.arctan((y2 - y1) / (x2 - x1))
            #
            # hdx = 0.5 * face_width * (2. - np.abs(np.cos(roll)))
            # hdy = 0.5 * face_width * (1. + np.abs(np.sin(roll)))
            # print(np.degrees(roll), face_width, hdx, hdy)
            # frame['faces'][i] = (int(cx - hdx), int(cy - hdy), int(2*hdx), int(2*hdy))

_face_detector = None
_landmarks_predictor = None


def _get_dlib_data_file(dat_name):
    dat_dir = os.path.relpath('%s/../3rdparty' % dir)
    dat_path = '%s/%s' % (dat_dir, dat_name)
    if not os.path.isdir(dat_dir):
        os.mkdir(dat_dir)

    # Download trained shape detector
    if not os.path.isfile(dat_path):
        with urlopen('http://dlib.net/files/%s.bz2' % dat_name) as response:
            with bz2.BZ2File(response) as bzf, open(dat_path, 'wb') as f:
                shutil.copyfileobj(bzf, f)

    return dat_path


def _get_opencv_xml(xml_name):
    xml_dir = os.path.relpath('%s/../3rdparty' % dir ) ##os.path.basename(__file__)
    xml_path = '%s/%s' % (xml_dir, xml_name)
    if not os.path.isdir(xml_dir):
        os.mkdir(xml_dir)

    # Download trained shape detector
    if not os.path.isfile(xml_path):
        url_stem = 'https://raw.githubusercontent.com/opencv/opencv/master/data/lbpcascades'
        with urlopen('%s/%s' % (url_stem, xml_name)) as response:
            with open(xml_path, 'wb') as f:
                shutil.copyfileobj(response, f)

    return xml_path


def get_face_detector():
    """Get a singleton dlib face detector."""
    global _face_detector
    if not _face_detector:
        try:
            dat_path = _get_dlib_data_file('mmod_human_face_detector.dat')
            _face_detector = dlib.cnn_face_detection_model_v1(dat_path)
        except:
            xml_path = _get_opencv_xml('lbpcascade_frontalface_improved.xml')
            _face_detector = cv.CascadeClassifier(xml_path)
    return _face_detector


def get_landmarks_predictor():
    """Get a singleton dlib face landmark predictor."""
    global _landmarks_predictor
    if not _landmarks_predictor:
        dat_path = _get_dlib_data_file('shape_predictor_5_face_landmarks.dat')
        # dat_path = _get_dlib_data_file('shape_predictor_68_face_landmarks.dat')
        _landmarks_predictor = dlib.shape_predictor(dat_path)
    return _landmarks_predictor

In [ ]:
##gaze


"""Utility methods for gaze angle and error calculations."""
import cv2 as cv
import numpy as np
import tensorflow as tf


def pitchyaw_to_vector(pitchyaws):
    r"""Convert given yaw (:math:`\theta`) and pitch (:math:`\phi`) angles to unit gaze vectors.
    Args:
        pitchyaws (:obj:`numpy.array`): yaw and pitch angles :math:`(n\times 2)` in radians.
    Returns:
        :obj:`numpy.array` of shape :math:`(n\times 3)` with 3D vectors per row.
    """
    n = pitchyaws.shape[0]
    sin = np.sin(pitchyaws)
    cos = np.cos(pitchyaws)
    out = np.empty((n, 3))
    out[:, 0] = np.multiply(cos[:, 0], sin[:, 1])
    out[:, 1] = sin[:, 0]
    out[:, 2] = np.multiply(cos[:, 0], cos[:, 1])
    return out


def vector_to_pitchyaw(vectors):
    r"""Convert given gaze vectors to yaw (:math:`\theta`) and pitch (:math:`\phi`) angles.
    Args:
        vectors (:obj:`numpy.array`): gaze vectors in 3D :math:`(n\times 3)`.
    Returns:
        :obj:`numpy.array` of shape :math:`(n\times 2)` with values in radians.
    """
    n = vectors.shape[0]
    out = np.empty((n, 2))
    vectors = np.divide(vectors, np.linalg.norm(vectors, axis=1).reshape(n, 1))
    out[:, 0] = np.arcsin(vectors[:, 1])  # theta
    out[:, 1] = np.arctan2(vectors[:, 0], vectors[:, 2])  # phi
    return out

radians_to_degrees = 180.0 / np.pi


def angular_error(a, b):
    """Calculate angular error (via cosine similarity)."""
    a = pitchyaw_to_vector(a) if a.shape[1] == 2 else a
    b = pitchyaw_to_vector(b) if b.shape[1] == 2 else b

    ab = np.sum(np.multiply(a, b), axis=1)
    a_norm = np.linalg.norm(a, axis=1)
    b_norm = np.linalg.norm(b, axis=1)

    # Avoid zero-values (to avoid NaNs)
    a_norm = np.clip(a_norm, a_min=1e-7, a_max=None)
    b_norm = np.clip(b_norm, a_min=1e-7, a_max=None)

    similarity = np.divide(ab, np.multiply(a_norm, b_norm))

    return np.arccos(similarity) * radians_to_degrees


def mean_angular_error(a, b):
    """Calculate mean angular error (via cosine similarity)."""
    return np.mean(angular_error(a, b))


def tensorflow_angular_error_from_pitchyaw(y_true, y_pred):
    """Tensorflow method to calculate angular loss from head angles."""
    def angles_to_unit_vectors(y):
        sin = tf.sin(y)
        cos = tf.cos(y)
        return tf.stack([
            tf.multiply(cos[:, 0], sin[:, 1]),
            sin[:, 0],
            tf.multiply(cos[:, 0], cos[:, 1]),
        ], axis=1)

    with tf.name_scope('mean_angular_error'):
        v_true = angles_to_unit_vectors(y_true)
        v_pred = angles_to_unit_vectors(y_pred)
        return tensorflow_angular_error_from_vector(v_true, v_pred)


def tensorflow_angular_error_from_vector(v_true, v_pred):
    """Tensorflow method to calculate angular loss from 3D vector."""
    with tf.name_scope('mean_angular_error'):
        v_true_norm = tf.sqrt(tf.reduce_sum(tf.square(v_true), axis=1))
        v_pred_norm = tf.sqrt(tf.reduce_sum(tf.square(v_pred), axis=1))

        sim = tf.div(tf.reduce_sum(tf.multiply(v_true, v_pred), axis=1),
                     tf.multiply(v_true_norm, v_pred_norm))

        # Floating point precision can cause sim values to be slightly outside of
        # [-1, 1] so we clip values
        sim = tf.clip_by_value(sim, -1.0 + 1e-6, 1.0 - 1e-6)

        ang = tf.scalar_mul(radians_to_degrees, tf.acos(sim))
        return tf.reduce_mean(ang)


def draw_gaze(image_in, eye_pos, pitchyaw, length=40.0, thickness=3, color=(0, 255,0)):
    """Draw gaze angle on given image with a given eye positions."""
    image_out = image_in
    if len(image_out.shape) == 2 or image_out.shape[2] == 1:
        image_out = cv.cvtColor(image_out, cv.COLOR_GRAY2BGR)
    dx = -length * np.sin(pitchyaw[1])
    dy = -length * np.sin(pitchyaw[0])
    cv.arrowedLine(image_out, tuple(np.round(eye_pos).astype(np.int32)),
                   tuple(np.round([eye_pos[0] + dx, eye_pos[1] + dy]).astype(int)), color,
                   thickness, cv.LINE_AA, tipLength=0.2)
    return image_out

In [ ]:
##gazemap 

"""Utility methods for generating gazemaps."""
import cv2 as cv
import numpy as np

height_to_eyeball_radius_ratio = 1.1
eyeball_radius_to_iris_diameter_ratio = 1.0

def from_gaze2d(gaze, output_size, scale=1.0):
    """Generate a normalized pictorial representation of 3D gaze direction."""
    gazemaps = []
    oh, ow = np.round(scale * np.asarray(output_size)).astype(np.int32)
    oh_2 = int(np.round(0.5 * oh))
    ow_2 = int(np.round(0.5 * ow))
    r = int(height_to_eyeball_radius_ratio * oh_2)
    theta, phi = gaze
    theta = -theta
    sin_theta = np.sin(theta)
    cos_theta = np.cos(theta)
    sin_phi = np.sin(phi)
    cos_phi = np.cos(phi)

    # Draw iris
    eyeball_radius = int(height_to_eyeball_radius_ratio * oh_2)
    iris_radius_angle = np.arcsin(0.5 * eyeball_radius_to_iris_diameter_ratio)
    iris_radius = eyeball_radius_to_iris_diameter_ratio * eyeball_radius
    iris_distance = float(eyeball_radius) * np.cos(iris_radius_angle)
    iris_offset = np.asarray([
        -iris_distance * sin_phi * cos_theta,
        iris_distance * sin_theta,
    ])
    iris_centre = np.asarray([ow_2, oh_2]) + iris_offset
    angle = np.degrees(np.arctan2(iris_offset[1], iris_offset[0]))
    ellipse_max = eyeball_radius_to_iris_diameter_ratio * iris_radius
    ellipse_min = np.abs(ellipse_max * cos_phi * cos_theta)
    gazemap = np.zeros((oh, ow), dtype=np.float32)
    gazemap = cv.ellipse(gazemap, box=(iris_centre, (ellipse_min, ellipse_max), angle),
                         color=1.0, thickness=-1, lineType=cv.LINE_AA)
    gazemaps.append(gazemap)

    # Draw eyeball
    gazemap = np.zeros((oh, ow), dtype=np.float32)
    gazemap = cv.circle(gazemap, (ow_2, oh_2), r, color=1, thickness=-1)
    gazemaps.append(gazemap)

    return np.asarray(gazemaps)

In [ ]:
"""Utility methods for generating and visualizing heatmaps."""
import numpy as np


def gaussian_2d(shape, centre, sigma=1.0):
    """Generate heatmap with single 2D gaussian."""
    xs = np.arange(0.5, shape[1] + 0.5, step=1.0, dtype=np.float32)
    ys = np.expand_dims(np.arange(0.5, shape[0] + 0.5, step=1.0, dtype=np.float32), -1)
    alpha = -0.5 / (sigma**2)
    heatmap = np.exp(alpha * ((xs - centre[0])**2 + (ys - centre[1])**2))
    return heatmap

In [ ]:
##hdf5


"""HDF5 data source for gaze estimation."""
from threading import Lock
from typing import List

import cv2 as cv
import h5py
import numpy as np
import tensorflow as tf

"""from core import BaseDataSource
import util.gazemap"""


class HDF5Source(BaseDataSource):
    """HDF5 data loading class (using h5py)."""

    def __init__(self,
                 tensorflow_session: tf.Session,
                 batch_size: int,
                 keys_to_use: List[str],
                 hdf_path: str,
                 testing=False,
                 eye_image_shape=(36, 60),
                 **kwargs):
        """Create queues and threads to read and preprocess data from specified keys."""
        hdf5 = h5py.File(hdf_path, 'r')
        self._short_name = 'HDF:%s' % '/'.join(hdf_path.split('/')[-2:])
        if testing:
            self._short_name += ':test'

        # Cache some parameters
        self._eye_image_shape = eye_image_shape

        # Create global index over all specified keys
        self._index_to_key = {}
        index_counter = 0
        for key in keys_to_use:
            n = hdf5[key]['eye'].shape[0]
            for i in range(n):
                self._index_to_key[index_counter] = (key, i)
                index_counter += 1
        self._num_entries = index_counter

        self._hdf5 = hdf5
        self._mutex = Lock()
        self._current_index = 0
        super().__init__(tensorflow_session, batch_size=batch_size, testing=testing, **kwargs)

        # Set index to 0 again as base class constructor called HDF5Source::entry_generator once to
        # get preprocessed sample.
        self._current_index = 0

    @property
    def num_entries(self):
        """Number of entries in this data source."""
        return self._num_entries

    @property
    def short_name(self):
        """Short name specifying source HDF5."""
        return self._short_name

    def cleanup(self):
        """Close HDF5 file before running base class cleanup routine."""
        super().cleanup()

    def reset(self):
        """Reset index."""
        with self._mutex:
            super().reset()
            self._current_index = 0

    def entry_generator(self, yield_just_one=False):
        """Read entry from HDF5."""
        try:
            while range(1) if yield_just_one else True:
                with self._mutex:
                    if self._current_index >= self.num_entries:
                        if self.testing:
                            break
                        else:
                            self._current_index = 0
                    current_index = self._current_index
                    self._current_index += 1

                key, index = self._index_to_key[current_index]
                data = self._hdf5[key]
                entry = {}
                for name in ('eye', 'gaze', 'head'):
                    if name in data:
                        entry[name] = data[name][index, :]
                yield entry
        finally:
            # Execute any cleanup operations as necessary
            pass

    def preprocess_entry(self, entry):
        """Resize eye image and normalize intensities."""
        oh, ow = self._eye_image_shape
        eye = entry['eye']
        eye = cv.resize(eye, (ow, oh))
        eye = eye.astype(np.float32)
        eye *= 2.0 / 255.0
        eye -= 1.0
        eye = np.expand_dims(eye, axis=0 if self.data_format == 'NCHW' else -1)
        entry['eye'] = eye

        entry['gazemaps'] = from_gaze2d( ##util.gazemap.from_gaze2d(
            entry['gaze'], output_size=(oh, ow), scale=0.5,
        ).astype(np.float32)
        if self.data_format == 'NHWC':
            np.transpose(entry['gazemaps'], (1, 2, 0))

        # Ensure all values in an entry are 4-byte floating point numbers
        for key, value in entry.items():
            entry[key] = value.astype(np.float32)

        return entry

In [ ]:
##unityeyes

"""UnityEyes data source for gaze estimation."""
import os
from threading import Lock

import cv2 as cv
import numpy as np
import tensorflow as tf
import ujson

"""from core import BaseDataSource
import util.gaze
import util.heatmap"""


class UnityEyes(BaseDataSource):
    """UnityEyes data loading class."""

    def __init__(self,
                 tensorflow_session: tf.Session,
                 batch_size: int,
                 unityeyes_path: str,
                 testing=False,
                 generate_heatmaps=False,
                 eye_image_shape=(36, 60),
                 heatmaps_scale=1.0,
                 **kwargs):
        """Create queues and threads to read and preprocess data."""
        self._short_name = 'UnityEyes'
        if testing:
            self._short_name += ':test'

        # Cache some parameters
        self._eye_image_shape = eye_image_shape
        self._heatmaps_scale = heatmaps_scale

        # Create global index over all specified keys
        self._images_path = unityeyes_path
        self._file_stems = sorted([p[:-5] for p in os.listdir(unityeyes_path)
                                   if p.endswith('.json')])
        self._num_entries = len(self._file_stems)
        self._num_entries = self._num_entries//10

        self._mutex = Lock()
        self._current_index = 0

        # Define bounds for noise values for different augmentation types
        self._difficulty = 0.0
        self._augmentation_ranges = {  # (easy, hard)
            'translation': (2.0, 10.0),
            'rotation': (0.1, 2.0),
            'intensity': (0.5, 20.0),
            'blur': (0.1, 1.0),
            'scale': (0.01, 0.1),
            'rescale': (1.0, 0.2),
            'num_line': (0.0, 2.0),
            'heatmap_sigma': (5.0, 2.5),
        }
        self._generate_heatmaps = generate_heatmaps

        # Call parent class constructor
        super().__init__(tensorflow_session, batch_size=batch_size, testing=testing, **kwargs)

    @property
    def num_entries(self):
        """Number of entries in this data source."""
        return self._num_entries

    @property
    def short_name(self):
        """Short name specifying source UnityEyes."""
        return self._short_name

    def reset(self):
        """Reset index."""
        with self._mutex:
            super().reset()
            self._current_index = 0

    def entry_generator(self, yield_just_one=False):
        """Read entry from UnityEyes."""
        try:
            while range(1) if yield_just_one else True:
                with self._mutex:
                    if self._current_index >= self.num_entries:
                        if self.testing:
                            break
                        else:
                            self._current_index = 0
                    current_index = self._current_index
                    self._current_index += 1

                file_stem = self._file_stems[current_index]
                jpg_path = '%s/%s.jpg' % (self._images_path, file_stem)
                json_path = '%s/%s.json' % (self._images_path, file_stem)
                if not os.path.isfile(jpg_path) or not os.path.isfile(json_path):
                    continue
                with open(json_path, 'r') as f:
                    json_data = ujson.load(f)
                entry = {
                    'full_image': cv.imread(jpg_path, cv.IMREAD_GRAYSCALE),
                    'json_data': json_data,
                }
                assert entry['full_image'] is not None
                yield entry
        finally:
            # Execute any cleanup operations as necessary
            pass

    def set_difficulty(self, difficulty):
        """Set difficulty of training data."""
        assert isinstance(difficulty, float)
        assert 0.0 <= difficulty <= 1.0
        self._difficulty = difficulty

    def set_augmentation_range(self, augmentation_type, easy_value, hard_value):
        """Set 'range' for a known augmentation type."""
        assert isinstance(augmentation_type, str)
        assert augmentation_type in self._augmentation_ranges
        assert isinstance(easy_value, float) or isinstance(easy_value, int)
        assert isinstance(hard_value, float) or isinstance(hard_value, int)
        self._augmentation_ranges[augmentation_type] = (easy_value, hard_value)

    def preprocess_entry(self, entry):
        """Use annotations to segment eyes and calculate gaze direction."""
        full_image = entry['full_image']
        json_data = entry['json_data']
        del entry['full_image']
        del entry['json_data']

        ih, iw = full_image.shape
        iw_2, ih_2 = 0.5 * iw, 0.5 * ih
        oh, ow = self._eye_image_shape

        def process_coords(coords_list):
            coords = [eval(l) for l in coords_list]
            return np.array([(x, ih-y, z) for (x, y, z) in coords])
        interior_landmarks = process_coords(json_data['interior_margin_2d'])
        caruncle_landmarks = process_coords(json_data['caruncle_2d'])
        iris_landmarks = process_coords(json_data['iris_2d'])

        random_multipliers = []

        def value_from_type(augmentation_type):
            # Scale to be in range
            easy_value, hard_value = self._augmentation_ranges[augmentation_type]
            value = (hard_value - easy_value) * self._difficulty + easy_value
            value = (np.clip(value, easy_value, hard_value)
                     if easy_value < hard_value
                     else np.clip(value, hard_value, easy_value))
            return value

        def noisy_value_from_type(augmentation_type):
            # Get normal distributed random value
            if len(random_multipliers) == 0:
                random_multipliers.extend(
                        list(np.random.normal(size=(len(self._augmentation_ranges),))))
            return random_multipliers.pop() * value_from_type(augmentation_type)

        # Only select almost frontal images
        h_pitch, h_yaw, _ = eval(json_data['head_pose'])
        if h_pitch > 180.0:  # Need to correct pitch
            h_pitch -= 360.0
        h_yaw -= 180.0  # Need to correct yaw
        if abs(h_pitch) > 20 or abs(h_yaw) > 20:
            return None

        # Prepare to segment eye image
        left_corner = np.mean(caruncle_landmarks[:, :2], axis=0)
        right_corner = interior_landmarks[8, :2]
        eye_width = 1.5 * abs(left_corner[0] - right_corner[0])
        eye_middle = np.mean([np.amin(interior_landmarks[:, :2], axis=0),
                              np.amax(interior_landmarks[:, :2], axis=0)], axis=0)

        # Centre axes to eyeball centre
        translate_mat = np.asmatrix(np.eye(3))
        translate_mat[:2, 2] = [[-iw_2], [-ih_2]]

        # Rotate eye image if requested
        rotate_mat = np.asmatrix(np.eye(3))
        rotation_noise = noisy_value_from_type('rotation')
        if rotation_noise > 0:
            rotate_angle = np.radians(rotation_noise)
            cos_rotate = np.cos(rotate_angle)
            sin_rotate = np.sin(rotate_angle)
            rotate_mat[0, 0] = cos_rotate
            rotate_mat[0, 1] = -sin_rotate
            rotate_mat[1, 0] = sin_rotate
            rotate_mat[1, 1] = cos_rotate

        # Scale image to fit output dimensions (with a little bit of noise)
        scale_mat = np.asmatrix(np.eye(3))
        scale = 1. + noisy_value_from_type('scale')
        scale_inv = 1. / scale
        np.fill_diagonal(scale_mat, ow / eye_width * scale)
        original_eyeball_radius = 71.7593
        eyeball_radius = original_eyeball_radius * scale_mat[0, 0]  # See: https://goo.gl/ZnXgDE
        entry['radius'] = np.float32(eyeball_radius)

        # Re-centre eye image such that eye fits (based on determined `eye_middle`)
        recentre_mat = np.asmatrix(np.eye(3))
        recentre_mat[0, 2] = iw/2 - eye_middle[0] + 0.5 * eye_width * scale_inv
        recentre_mat[1, 2] = ih/2 - eye_middle[1] + 0.5 * oh / ow * eye_width * scale_inv
        recentre_mat[0, 2] += noisy_value_from_type('translation')  # x
        recentre_mat[1, 2] += noisy_value_from_type('translation')  # y

        # Apply transforms
        transform_mat = recentre_mat * scale_mat * rotate_mat * translate_mat
        eye = cv.warpAffine(full_image, transform_mat[:2, :3], (ow, oh))

        # Convert look vector to gaze direction in polar angles
        look_vec = np.array(eval(json_data['eye_details']['look_vec']))[:3]
        look_vec[0] = -look_vec[0]
        original_gaze = vector_to_pitchyaw(look_vec.reshape((1, 3))).flatten() ##util.gaze.
        look_vec = rotate_mat * look_vec.reshape(3, 1)
        gaze = vector_to_pitchyaw(look_vec.reshape((1, 3))).flatten() ##util.gaze.
        if gaze[1] > 0.0:
            gaze[1] = np.pi - gaze[1]
        elif gaze[1] < 0.0:
            gaze[1] = -(np.pi + gaze[1])
        entry['gaze'] = gaze.astype(np.float32)

        # Draw line randomly
        num_line_noise = int(np.round(noisy_value_from_type('num_line')))
        if num_line_noise > 0:
            line_rand_nums = np.random.rand(5 * num_line_noise)
            for i in range(num_line_noise):
                j = 5 * i
                lx0, ly0 = int(ow * line_rand_nums[j]), oh
                lx1, ly1 = ow, int(oh * line_rand_nums[j + 1])
                direction = line_rand_nums[j + 2]
                if direction < 0.25:
                    lx1 = ly0 = 0
                elif direction < 0.5:
                    lx1 = 0
                elif direction < 0.75:
                    ly0 = 0
                line_colour = int(255 * line_rand_nums[j + 3])
                eye = cv.line(eye, (lx0, ly0), (lx1, ly1),
                              color=(line_colour, line_colour, line_colour),
                              thickness=max(1, int(6*line_rand_nums[j + 4])),
                              lineType=cv.LINE_AA)

        # Rescale image if required
        rescale_max = value_from_type('rescale')
        if rescale_max < 1.0:
            rescale_noise = np.random.uniform(low=rescale_max, high=1.0)
            interpolation = cv.INTER_CUBIC
            eye = cv.resize(eye, dsize=(0, 0), fx=rescale_noise, fy=rescale_noise,
                            interpolation=interpolation)
            eye = cv.equalizeHist(eye)
            eye = cv.resize(eye, dsize=(ow, oh), interpolation=interpolation)

        # Add rgb noise to eye image
        intensity_noise = int(value_from_type('intensity'))
        if intensity_noise > 0:
            eye = eye.astype(np.int16)
            eye += np.random.randint(low=-intensity_noise, high=intensity_noise,
                                     size=eye.shape, dtype=np.int16)
            cv.normalize(eye, eye, alpha=0, beta=255, norm_type=cv.NORM_MINMAX)
            eye = eye.astype(np.uint8)

        # Add blur to eye image
        blur_noise = noisy_value_from_type('blur')
        if blur_noise > 0:
            eye = cv.GaussianBlur(eye, (7, 7), 0.5 + np.abs(blur_noise))

        # Histogram equalization and preprocessing for NN
        eye = cv.equalizeHist(eye)
        eye = eye.astype(np.float32)
        eye *= 2.0 / 255.0
        eye -= 1.0
        eye = np.expand_dims(eye, -1 if self.data_format == 'NHWC' else 0)
        entry['eye'] = eye

        # Select and transform landmark coordinates
        iris_centre = np.asarray([
            iw_2 + original_eyeball_radius * -np.cos(original_gaze[0]) * np.sin(original_gaze[1]),
            ih_2 + original_eyeball_radius * -np.sin(original_gaze[0]),
        ])
        landmarks = np.concatenate([interior_landmarks[::2, :2],  # 8
                                    iris_landmarks[::4, :2],  # 8
                                    iris_centre.reshape((1, 2)),
                                    [[iw_2, ih_2]],  # Eyeball centre
                                    ])  # 18 in total
        landmarks = np.asmatrix(np.pad(landmarks, ((0, 0), (0, 1)), 'constant',
                                       constant_values=1))
        landmarks = np.asarray(landmarks * transform_mat.T)
        landmarks = landmarks[:, :2]  # We only need x, y
        entry['landmarks'] = landmarks.astype(np.float32)

        # Generate heatmaps if necessary
        if self._generate_heatmaps:
            # Should be half-scale (compared to eye image). # util.heatmap.gaussian_2d
            entry['heatmaps'] = np.asarray([
                    gaussian_2d(
                    shape=(self._heatmaps_scale*oh, self._heatmaps_scale*ow),
                    centre=self._heatmaps_scale*landmark,
                    sigma=value_from_type('heatmap_sigma'),
                )
                for landmark in entry['landmarks']
            ]).astype(np.float32)
            if self.data_format == 'NHWC':
                entry['heatmaps'] = np.transpose(entry['heatmaps'], (1, 2, 0))

        return entry


In [ ]:
##video

"""Video (file) data source for gaze estimation."""
import os
import time

import cv2 as cv

##from .frames import FramesSource


class Video(FramesSource):
    """Video frame grabbing and preprocessing."""

    def __init__(self, video_path, **kwargs):
        """Create queues and threads to read and preprocess data."""
        self._short_name = 'Video'

        assert os.path.isfile(video_path)
        self._video_path = video_path
        self._capture = cv.VideoCapture(video_path)

        # Call parent class constructor
        super().__init__(staging=False, **kwargs)

    def frame_generator(self):
        """Read frame from webcam."""
        last_frame = None
        while True:
            ret, frame = self._capture.read()
            if ret:
                yield frame
                last_frame = frame
            else:
                yield last_frame
                break

    def frame_read_job(self):
        """Read frame from video (without skipping)."""
        generate_frame = self.frame_generator()
        while True:
            before_frame_read = time.time()
            try:
                bgr = next(generate_frame)
            except StopIteration:
                break
            if bgr is not None:
                after_frame_read = time.time()
                with self._read_mutex:
                    self._frame_read_queue.put((before_frame_read, bgr, after_frame_read))

        print('Video "%s" closed.' % self._video_path)
        self._open = False

In [ ]:
##train Deep Pictorial Gaze  model

"""Deep Pictorial Gaze architecture."""
from typing import Dict

import numpy as np
import scipy
import tensorflow as tf

"""from core import BaseDataSource, BaseModel
from datasources import UnityEyes
import util.gaze"""


class DPG(BaseModel):
    """Deep Pictorial Gaze architecture as introduced in [Park et al. ECCV'18]."""

    def __init__(self, tensorflow_session=None, first_layer_stride=2, num_modules=3,
                 num_feature_maps=32, growth_rate=8, extra_tags=[], **kwargs):
        """Specify DPG-specific parameters."""
        self._hg_first_layer_stride = first_layer_stride
        self._hg_num_modules = num_modules
        self._hg_num_feature_maps= num_feature_maps
        self._dn_growth_rate = growth_rate
        self._extra_tags = extra_tags

        # Call parent class constructor
        super().__init__(tensorflow_session, **kwargs)

    _hg_first_layer_stride = 2
    _hg_num_modules = 3
    _hg_num_feature_maps = 32
    _hg_num_residual_blocks = 1
    _hg_num_gazemaps = 2

    _dn_growth_rate = 8
    _dn_compression_factor = 0.5
    _dn_num_layers_per_block = (4, 4, 4, 4)
    _dn_num_dense_blocks = len(_dn_num_layers_per_block)

    @property
    def identifier(self):
        """Identifier for model based on data sources and parameters."""
        first_data_source = next(iter(self._train_data.values()))
        input_tensors = first_data_source.output_tensors
        if self._data_format == 'NHWC':
            _, eh, ew, _ = input_tensors['eye'].shape.as_list()
        else:
            _, _, eh, ew = input_tensors['eye'].shape.as_list()
        return 'DPG_i%dx%d_f%dx%d_n%d_m%d_k%d_%s' % (
            ew, eh,
            int(ew / self._hg_first_layer_stride),
            int(eh / self._hg_first_layer_stride),
            self._hg_num_feature_maps, self._hg_num_modules,
            self._dn_growth_rate,
            '-'.join(self._extra_tags) if len(self._extra_tags) > 0 else '',
        )

    def train_loop_pre(self, current_step):
        """Run this at beginning of training loop."""
        # Step learning rate decay
        multiplier = np.power(0.1, int(current_step / 10000))
        self._tensorflow_session.run(self.assign_learning_rate_multiplier, feed_dict={
            self.learning_rate_multiplier_placeholder: multiplier,
        })

    _column_of_ones = None
    _column_of_zeros = None

    def _augment_training_images(self, images, mode):
        if mode == 'test':
            return images
        with tf.variable_scope('augment'):
            if self._data_format == 'NCHW':
                images = tf.transpose(images, perm=[0, 2, 3, 1])
            n, h, w, _ = images.shape.as_list()
            if self._column_of_ones is None:
                self._column_of_ones = tf.ones((n, 1))
                self._column_of_zeros = tf.zeros((n, 1))
            transforms = tf.concat([
                self._column_of_ones,
                self._column_of_zeros,
                tf.truncated_normal((n, 1), mean=0, stddev=.05*w),
                self._column_of_zeros,
                self._column_of_ones,
                tf.truncated_normal((n, 1), mean=0, stddev=.05*h),
                self._column_of_zeros,
                self._column_of_zeros,
            ], axis=1)
            images = tf.contrib.image.transform(images, transforms, interpolation='BILINEAR')
            if self._data_format == 'NCHW':
                images = tf.transpose(images, perm=[0, 3, 1, 2])
        return images

    def build_model(self, data_sources: Dict[str, BaseDataSource], mode: str):
        """Build model."""
        data_source = next(iter(data_sources.values()))
        input_tensors = data_source.output_tensors
        x = input_tensors['eye']
        y1 = input_tensors['gazemaps'] if 'gazemaps' in input_tensors else None
        y2 = input_tensors['gaze'] if 'gaze' in input_tensors else None

        with tf.variable_scope('input_data'):
            # self.summary.feature_maps('eyes', x, data_format=self._data_format_longer)
            if y1 is not None:
                self.summary.feature_maps('gazemaps', y1, data_format=self._data_format_longer)

        outputs = {}
        loss_terms = {}
        metrics = {}

        # Lightly augment training data
        x = self._augment_training_images(x, mode)

        with tf.variable_scope('hourglass'):
            # Prepare for Hourglass by downscaling via conv
            with tf.variable_scope('pre'):
                n = self._hg_num_feature_maps
                x = self._apply_conv(x, num_features=n, kernel_size=7,
                                     stride=self._hg_first_layer_stride)
                x = tf.nn.relu(self._apply_bn(x))
                x = self._build_residual_block(x, n, 2*n, name='res1')
                x = self._build_residual_block(x, 2*n, n, name='res2')

            # Hourglass blocks
            x_prev = x
            gmap = None
            for i in range(self._hg_num_modules):
                with tf.variable_scope('hg_%d' % (i + 1)):
                    x = self._build_hourglass(x, steps_to_go=4, num_features=self._hg_num_feature_maps)
                    x, gmap = self._build_hourglass_after(
                        x_prev, x, do_merge=(i < (self._hg_num_modules - 1)),
                    )
                    x_prev = x
            if y1 is not None:
                # Cross-entropy loss
                metrics['gazemaps_ce'] = -tf.reduce_mean(tf.reduce_sum(
                    y1 * tf.log(tf.clip_by_value(gmap, 1e-10, 1.0)),  # avoid NaN
                    axis=[1, 2, 3]))
                # metrics['gazemaps_ce'] = tf.losses.softmax_cross_entropy(
                #     tf.reshape(y1, (self._batch_size, -1)),
                #     tf.reshape(gmap, (self._batch_size, -1)),
                #     loss_collection=None,
                # )
            x = gmap
            outputs['gazemaps'] = gmap
            self.summary.feature_maps('bottleneck', gmap, data_format=self._data_format_longer)

        with tf.variable_scope('densenet'):
            # DenseNet blocks to regress to gaze
            for i in range(self._dn_num_dense_blocks):
                with tf.variable_scope('block%d' % (i + 1)):
                    x = self._apply_dense_block(x,
                                                num_layers=self._dn_num_layers_per_block[i])
                    if i == self._dn_num_dense_blocks - 1:
                        break
                with tf.variable_scope('trans%d' % (i + 1)):
                    x = self._apply_transition_layer(x)

            # Global average pooling
            with tf.variable_scope('post'):
                x = self._apply_bn(x)
                x = tf.nn.relu(x)
                if self._data_format == 'NCHW':
                    x = tf.reduce_mean(x, axis=[2, 3])
                else:
                    x = tf.reduce_mean(x, axis=[1, 2])
                x = tf.contrib.layers.flatten(x)

            # Output layer
            with tf.variable_scope('output'):
                x = self._apply_fc(x, 2)
                outputs['gaze'] = x
                if y2 is not None:
                    metrics['gaze_mse'] = tf.reduce_mean(tf.squared_difference(x, y2))
                    metrics['gaze_ang'] = tensorflow_angular_error_from_pitchyaw(y2, x) #util.gaze.tensorflow_angular_error_from_pitchyaw(y2, x)

        # Combine two loss terms
        if y1 is not None and y2 is not None:
            loss_terms['combined_loss'] = 1e-5*metrics['gazemaps_ce'] + metrics['gaze_mse']

        # Define outputs
        return outputs, loss_terms, metrics

    def _apply_conv(self, tensor, num_features, kernel_size=3, stride=1):
        return tf.layers.conv2d(
            tensor,
            num_features,
            kernel_size=kernel_size,
            strides=stride,
            padding='SAME',
            kernel_initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.01),
            kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-4),
            bias_initializer=tf.zeros_initializer(),
            data_format=self._data_format_longer,
            name='conv',
        )

    def _apply_fc(self, tensor, num_outputs):
        return tf.layers.dense(
            tensor,
            num_outputs,
            use_bias=True,
            kernel_initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.01),
            kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-4),
            bias_initializer=tf.zeros_initializer(),
            name='fc',
        )

    def _apply_pool(self, tensor, kernel_size=3, stride=2):
        tensor = tf.layers.max_pooling2d(
            tensor,
            pool_size=kernel_size,
            strides=stride,
            padding='SAME',
            data_format=self._data_format_longer,
            name='pool',
        )
        return tensor

    def _apply_bn(self, tensor):
        return tf.contrib.layers.batch_norm(
            tensor,
            scale=True,
            center=True,
            is_training=self.use_batch_statistics,
            trainable=True,
            data_format=self._data_format,
            updates_collections=None,
        )

    def _build_residual_block(self, x, num_in, num_out, name='res_block'):
        with tf.variable_scope(name):
            half_num_out = max(int(num_out/2), 1)
            c = x
            with tf.variable_scope('conv1'):
                c = tf.nn.relu(self._apply_bn(c))
                c = self._apply_conv(c, num_features=half_num_out, kernel_size=1, stride=1)
            with tf.variable_scope('conv2'):
                c = tf.nn.relu(self._apply_bn(c))
                c = self._apply_conv(c, num_features=half_num_out, kernel_size=3, stride=1)
            with tf.variable_scope('conv3'):
                c = tf.nn.relu(self._apply_bn(c))
                c = self._apply_conv(c, num_features=num_out, kernel_size=1, stride=1)
            with tf.variable_scope('skip'):
                if num_in == num_out:
                    s = tf.identity(x)
                else:
                    s = self._apply_conv(x, num_features=num_out, kernel_size=1, stride=1)
            x = c + s
        return x

    def _build_hourglass(self, x, steps_to_go, num_features, depth=1):
        with tf.variable_scope('depth%d' % depth):
            # Upper branch
            up1 = x
            for i in range(self._hg_num_residual_blocks):
                up1 = self._build_residual_block(up1, num_features, num_features,
                                                 name='up1_%d' % (i + 1))
            # Lower branch
            low1 = self._apply_pool(x, kernel_size=2, stride=2)
            for i in range(self._hg_num_residual_blocks):
                low1 = self._build_residual_block(low1, num_features, num_features,
                                                  name='low1_%d' % (i + 1))
            # Recursive
            low2 = None
            if steps_to_go > 1:
                low2 = self._build_hourglass(low1, steps_to_go - 1, num_features, depth=depth+1)
            else:
                low2 = low1
                for i in range(self._hg_num_residual_blocks):
                    low2 = self._build_residual_block(low2, num_features, num_features,
                                                      name='low2_%d' % (i + 1))
            # Additional residual blocks
            low3 = low2
            for i in range(self._hg_num_residual_blocks):
                low3 = self._build_residual_block(low3, num_features, num_features,
                                                  name='low3_%d' % (i + 1))
            # Upsample
            if self._data_format == 'NCHW':  # convert to NHWC
                low3 = tf.transpose(low3, (0, 2, 3, 1))
            up2 = tf.image.resize_bilinear(
                    low3,
                    up1.shape[1:3] if self._data_format == 'NHWC' else up1.shape[2:4],
                    align_corners=True,
                  )
            if self._data_format == 'NCHW':  # convert back from NHWC
                up2 = tf.transpose(up2, (0, 3, 1, 2))

        return up1 + up2

    def _build_hourglass_after(self, x_prev, x_now, do_merge=True):
        with tf.variable_scope('after'):
            for j in range(self._hg_num_residual_blocks):
                x_now = self._build_residual_block(x_now, self._hg_num_feature_maps,
                                                   self._hg_num_feature_maps,
                                                   name='after_hg_%d' % (j + 1))
            x_now = self._apply_conv(x_now, self._hg_num_feature_maps, kernel_size=1, stride=1)
            x_now = self._apply_bn(x_now)
            x_now = tf.nn.relu(x_now)

            with tf.variable_scope('gmap'):
                gmap = self._apply_conv(x_now, self._hg_num_gazemaps, kernel_size=1, stride=1)

        x_next = x_now
        if do_merge:
            with tf.variable_scope('merge'):
                with tf.variable_scope('gmap'):
                    x_gmaps = self._apply_conv(gmap, self._hg_num_feature_maps, kernel_size=1, stride=1)
                with tf.variable_scope('x'):
                    x_now = self._apply_conv(x_now, self._hg_num_feature_maps, kernel_size=1, stride=1)
                x_next += x_prev + x_gmaps

        # Perform softmax on gazemaps
        if self._data_format == 'NCHW':
            n, c, h, w = gmap.shape.as_list()
            gmap = tf.reshape(gmap, (n, -1))
            gmap = tf.nn.softmax(gmap)
            gmap = tf.reshape(gmap, (n, c, h, w))
        else:
            n, h, w, c = gmap.shape.as_list()
            gmap = tf.transpose(gmap, perm=[0, 3, 1, 2])
            gmap = tf.reshape(gmap, (n, -1))
            gmap = tf.nn.softmax(gmap)
            gmap = tf.reshape(gmap, (n, c, h, w))
            gmap = tf.transpose(gmap, perm=[0, 2, 3, 1])
        return x_next, gmap

    def _apply_dense_block(self, x, num_layers):
        assert isinstance(num_layers, int) and num_layers > 0
        c_index = 1 if self._data_format == 'NCHW' else 3
        x_prev = x
        for i in range(num_layers):
            with tf.variable_scope('layer%d' % (i + 1)):
                n = x.shape.as_list()[c_index]
                with tf.variable_scope('bottleneck'):
                    x = self._apply_composite_function(x,
                                                       num_features=min(n, 4*self._dn_growth_rate),
                                                       kernel_size=1)
                with tf.variable_scope('composite'):
                    x = self._apply_composite_function(x, num_features=self._dn_growth_rate,
                                                       kernel_size=3)
                if self._data_format == 'NCHW':
                    x = tf.concat([x, x_prev], axis=1)
                else:
                    x = tf.concat([x, x_prev], axis=-1)
                x_prev = x
        return x

    def _apply_transition_layer(self, x):
        c_index = 1 if self._data_format == 'NCHW' else 3
        x = self._apply_composite_function(
            x, num_features=int(self._dn_compression_factor * x.shape.as_list()[c_index]),
            kernel_size=1)
        x = tf.layers.average_pooling2d(x, pool_size=2, strides=2, padding='valid',
                                        data_format=self._data_format_longer)
        return x

    def _apply_composite_function(self, x, num_features=_dn_growth_rate, kernel_size=3):
        x = self._apply_bn(x)
        x = tf.nn.relu(x)
        x = self._apply_conv(x, num_features=num_features, kernel_size=kernel_size, stride=1)
        return x

In [ ]:


import argparse

import coloredlogs
import tensorflow as tf

def main():##if __name__ == '__main__':

    # Set global log level
    """parser = argparse.ArgumentParser(description='Train the Deep Pictorial Gaze model.')
    parser.add_argument('-v', type=str, help='logging level', default='info',
                        choices=['debug', 'info', 'warning', 'error', 'critical'])
    args = parser.parse_args()"""
    coloredlogs.install(
        datefmt='%d/%m %H:%M',
        fmt='%(asctime)s %(levelname)s %(message)s',
        level="info".upper() # args.v.upper(),
    )

    for i in range(0, 5):
        # Specify which people to train on, and which to test on
        person_id = 'p%02d' % i
        other_person_ids = ['p%02d' % j for j in range(5) if i != j]

        # Initialize Tensorflow session
        tf.reset_default_graph()
        tf.logging.set_verbosity(tf.logging.ERROR)
        gpu_options = tf.GPUOptions(allow_growth=True)
        with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as session:

            # Declare some parameters
            batch_size = 32

            # Define training data source
            #from datasources import HDF5Source

            # Define model
            #from models import DPG
            model = DPG(
                session,
                learning_schedule=[
                    {
                        'loss_terms_to_optimize': {
                            'combined_loss': ['hourglass', 'densenet'],
                        },
                        'metrics': ['gaze_mse', 'gaze_ang'],
                        'learning_rate': 0.0001, ##0.0002
                    },
                ],
                extra_tags=[person_id],

                # Data sources for training (and testing).
                train_data={
                    'mpi': HDF5Source(
                        session,
                        data_format='NCHW',
                        batch_size=batch_size,
                        keys_to_use=['train/' + s for s in other_person_ids],
                        hdf_path='/content/MPIIGaze.h5', ##'../datasets/MPIIGaze.h5',
                        eye_image_shape=(90, 150),
                        testing=False,
                        min_after_dequeue=30000,
                        staging=True,
                        shuffle=True,
                    ),
                },
                test_data={
                    'mpi': HDF5Source(
                        session,
                        data_format='NCHW',
                        batch_size=batch_size,
                        keys_to_use=['test/' + person_id],
                        hdf_path='/content/MPIIGaze.h5', ## '../datasets/MPIIGaze.h5',
                        eye_image_shape=(90, 150),
                        testing=True,
                    ),
                },
            )

            # Train this model for a set number of epochs
            model.train(
                num_epochs=20, ##20,
            )

            model.__del__()
            session.close()
            del session

In [ ]:
#main()

In [ ]:
"""Main script for gaze direction inference from webcam feed."""
import argparse
import os
import queue
import threading
import time

import coloredlogs
import cv2 as cv
import numpy as np
import tensorflow as tf

##from datasources import Video, Webcam
##from models import ELG
##import util.gaze

#if __name__ == '__main__':
def mainTestModelDPG():
    # Set global log level
    parser = argparse.ArgumentParser(description='Demonstration of landmarks localization.')
    parser.add_argument('-v', type=str, help='logging level', default='info',
                        choices=['debug', 'info', 'warning', 'error', 'critical'])
    parser.add_argument('--from_video', type=str, help='Use this video path instead of webcam')
    parser.add_argument('--record_video', type=str, help='Output path of video of demonstration.')
    parser.add_argument('--fullscreen', action='store_true')
    parser.add_argument('--headless', action='store_true')

    parser.add_argument('--fps', type=int, default=60, help='Desired sampling rate of webcam')
    parser.add_argument('--camera_id', type=int, default=0, help='ID of webcam to use')

    args = parser.parse_args()
    coloredlogs.install(
        datefmt='%d/%m %H:%M',
        fmt='%(asctime)s %(levelname)s %(message)s',
        level=args.v.upper(),
    )

    fps = 60
    record_video = dir + "/outputVideo/"
    from_video = dir + "/inputVideo/"


    # Check if GPU is available
    from tensorflow.python.client import device_lib
    session_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
    gpu_available = False
    try:
        gpus = [d for d in device_lib.list_local_devices(config=session_config)
                if d.device_type == 'GPU']
        gpu_available = len(gpus) > 0
    except:
        pass

    # Initialize Tensorflow session
    tf.logging.set_verbosity(tf.logging.INFO)
    with tf.Session(config=session_config) as session:

        # Declare some parameters
        batch_size = 2

        # Define webcam stream data source
        # Change data_format='NHWC' if not using CUDA
        if from_video:
            assert os.path.isfile(from_video)
            data_source = Video(from_video,
                                tensorflow_session=session, batch_size=batch_size,
                                data_format='NCHW' if gpu_available else 'NHWC',
                                eye_image_shape=(108, 180))
        """else:
            data_source = Webcam(tensorflow_session=session, batch_size=batch_size,
                                 camera_id=args.camera_id, fps=args.fps,
                                 data_format='NCHW' if gpu_available else 'NHWC',
                                 eye_image_shape=(36, 60))"""

        # Define model
        if args.from_video:
            model = ELG(
                session, train_data={'videostream': data_source},
                first_layer_stride=3,
                num_modules=3,
                num_feature_maps=64,
                learning_schedule=[
                    {
                        'loss_terms_to_optimize': {'dummy': ['hourglass', 'radius']},
                    },
                ],
            )
        else:
            model = ELG(
                session, train_data={'videostream': data_source},
                first_layer_stride=1,
                num_modules=2,
                num_feature_maps=32,
                learning_schedule=[
                    {
                        'loss_terms_to_optimize': {'dummy': ['hourglass', 'radius']},
                    },
                ],
            )

        # Record output frames to file if requested
        if args.record_video:
            video_out = None
            video_out_queue = queue.Queue()
            video_out_should_stop = False
            video_out_done = threading.Condition()

            def _record_frame():
                global video_out
                last_frame_time = None
                out_fps = 30
                out_frame_interval = 1.0 / out_fps
                while not video_out_should_stop:
                    frame_index = video_out_queue.get()
                    if frame_index is None:
                        break
                    assert frame_index in data_source._frames
                    frame = data_source._frames[frame_index]['bgr']
                    h, w, _ = frame.shape
                    if video_out is None:
                        video_out = cv.VideoWriter(
                            args.record_video, cv.VideoWriter_fourcc(*'H264'),
                            out_fps, (w, h),
                        )
                    now_time = time.time()
                    if last_frame_time is not None:
                        time_diff = now_time - last_frame_time
                        while time_diff > 0.0:
                            video_out.write(frame)
                            time_diff -= out_frame_interval
                    last_frame_time = now_time
                video_out.release()
                with video_out_done:
                    video_out_done.notify_all()
            record_thread = threading.Thread(target=_record_frame, name='record')
            record_thread.daemon = True
            record_thread.start()

        # Begin visualization thread
        inferred_stuff_queue = queue.Queue()

        def _visualize_output():
            last_frame_index = 0
            last_frame_time = time.time()
            fps_history = []
            all_gaze_histories = []

            if args.fullscreen:
                cv.namedWindow('vis', cv.WND_PROP_FULLSCREEN)
                cv.setWindowProperty('vis', cv.WND_PROP_FULLSCREEN, cv.WINDOW_FULLSCREEN)

            while True:
                # If no output to visualize, show unannotated frame
                if inferred_stuff_queue.empty():
                    next_frame_index = last_frame_index + 1
                    if next_frame_index in data_source._frames:
                        next_frame = data_source._frames[next_frame_index]
                        if 'faces' in next_frame and len(next_frame['faces']) == 0:
                            if not args.headless:
                                cv.imshow('vis', next_frame['bgr'])
                            if args.record_video:
                                video_out_queue.put_nowait(next_frame_index)
                            last_frame_index = next_frame_index
                    if cv.waitKey(1) & 0xFF == ord('q'):
                        return
                    continue

                # Get output from neural network and visualize
                output = inferred_stuff_queue.get()
                bgr = None
                for j in range(batch_size):
                    frame_index = output['frame_index'][j]
                    if frame_index not in data_source._frames:
                        continue
                    frame = data_source._frames[frame_index]

                    # Decide which landmarks are usable
                    heatmaps_amax = np.amax(output['heatmaps'][j, :].reshape(-1, 18), axis=0)
                    can_use_eye = np.all(heatmaps_amax > 0.7)
                    can_use_eyelid = np.all(heatmaps_amax[0:8] > 0.75)
                    can_use_iris = np.all(heatmaps_amax[8:16] > 0.8)

                    start_time = time.time()
                    eye_index = output['eye_index'][j]
                    bgr = frame['bgr']
                    eye = frame['eyes'][eye_index]
                    eye_image = eye['image']
                    eye_side = eye['side']
                    eye_landmarks = output['landmarks'][j, :]
                    eye_radius = output['radius'][j][0]
                    if eye_side == 'left':
                        eye_landmarks[:, 0] = eye_image.shape[1] - eye_landmarks[:, 0]
                        eye_image = np.fliplr(eye_image)

                    # Embed eye image and annotate for picture-in-picture
                    eye_upscale = 2
                    eye_image_raw = cv.cvtColor(cv.equalizeHist(eye_image), cv.COLOR_GRAY2BGR)
                    eye_image_raw = cv.resize(eye_image_raw, (0, 0), fx=eye_upscale, fy=eye_upscale)
                    eye_image_annotated = np.copy(eye_image_raw)
                    if can_use_eyelid:
                        cv.polylines(
                            eye_image_annotated,
                            [np.round(eye_upscale*eye_landmarks[0:8]).astype(np.int32)
                                                                     .reshape(-1, 1, 2)],
                            isClosed=True, color=(255, 255, 0), thickness=1, lineType=cv.LINE_AA,
                        )
                    if can_use_iris:
                        cv.polylines(
                            eye_image_annotated,
                            [np.round(eye_upscale*eye_landmarks[8:16]).astype(np.int32)
                                                                      .reshape(-1, 1, 2)],
                            isClosed=True, color=(0, 255, 255), thickness=1, lineType=cv.LINE_AA,
                        )
                        cv.drawMarker(
                            eye_image_annotated,
                            tuple(np.round(eye_upscale*eye_landmarks[16, :]).astype(np.int32)),
                            color=(0, 255, 255), markerType=cv.MARKER_CROSS, markerSize=4,
                            thickness=1, line_type=cv.LINE_AA,
                        )
                    face_index = int(eye_index / 2)
                    eh, ew, _ = eye_image_raw.shape
                    v0 = face_index * 2 * eh
                    v1 = v0 + eh
                    v2 = v1 + eh
                    u0 = 0 if eye_side == 'left' else ew
                    u1 = u0 + ew
                    bgr[v0:v1, u0:u1] = eye_image_raw
                    bgr[v1:v2, u0:u1] = eye_image_annotated

                    # Visualize preprocessing results
                    frame_landmarks = (frame['smoothed_landmarks']
                                       if 'smoothed_landmarks' in frame
                                       else frame['landmarks'])
                    for f, face in enumerate(frame['faces']):
                        for landmark in frame_landmarks[f][:-1]:
                            cv.drawMarker(bgr, tuple(np.round(landmark).astype(np.int32)),
                                          color=(0, 0, 255), markerType=cv.MARKER_STAR,
                                          markerSize=2, thickness=1, line_type=cv.LINE_AA)
                        cv.rectangle(
                            bgr, tuple(np.round(face[:2]).astype(np.int32)),
                            tuple(np.round(np.add(face[:2], face[2:])).astype(np.int32)),
                            color=(0, 255, 255), thickness=1, lineType=cv.LINE_AA,
                        )

                    # Transform predictions
                    eye_landmarks = np.concatenate([eye_landmarks,
                                                    [[eye_landmarks[-1, 0] + eye_radius,
                                                      eye_landmarks[-1, 1]]]])
                    eye_landmarks = np.asmatrix(np.pad(eye_landmarks, ((0, 0), (0, 1)),
                                                       'constant', constant_values=1.0))
                    eye_landmarks = (eye_landmarks *
                                     eye['inv_landmarks_transform_mat'].T)[:, :2]
                    eye_landmarks = np.asarray(eye_landmarks)
                    eyelid_landmarks = eye_landmarks[0:8, :]
                    iris_landmarks = eye_landmarks[8:16, :]
                    iris_centre = eye_landmarks[16, :]
                    eyeball_centre = eye_landmarks[17, :]
                    eyeball_radius = np.linalg.norm(eye_landmarks[18, :] -
                                                    eye_landmarks[17, :])

                    # Smooth and visualize gaze direction
                    num_total_eyes_in_frame = len(frame['eyes'])
                    if len(all_gaze_histories) != num_total_eyes_in_frame:
                        all_gaze_histories = [list() for _ in range(num_total_eyes_in_frame)]
                    gaze_history = all_gaze_histories[eye_index]
                    if can_use_eye:
                        # Visualize landmarks
                        cv.drawMarker(  # Eyeball centre
                            bgr, tuple(np.round(eyeball_centre).astype(np.int32)),
                            color=(0, 255, 0), markerType=cv.MARKER_CROSS, markerSize=4,
                            thickness=1, line_type=cv.LINE_AA,
                        )
                        # cv.circle(  # Eyeball outline
                        #     bgr, tuple(np.round(eyeball_centre).astype(np.int32)),
                        #     int(np.round(eyeball_radius)), color=(0, 255, 0),
                        #     thickness=1, lineType=cv.LINE_AA,
                        # )

                        # Draw "gaze"
                        # from models.elg import estimate_gaze_from_landmarks
                        # current_gaze = estimate_gaze_from_landmarks(
                        #     iris_landmarks, iris_centre, eyeball_centre, eyeball_radius)
                        i_x0, i_y0 = iris_centre
                        e_x0, e_y0 = eyeball_centre
                        theta = -np.arcsin(np.clip((i_y0 - e_y0) / eyeball_radius, -1.0, 1.0))
                        phi = np.arcsin(np.clip((i_x0 - e_x0) / (eyeball_radius * -np.cos(theta)),
                                                -1.0, 1.0))
                        current_gaze = np.array([theta, phi])
                        gaze_history.append(current_gaze)
                        gaze_history_max_len = 10
                        if len(gaze_history) > gaze_history_max_len:
                            gaze_history = gaze_history[-gaze_history_max_len:]
                        draw_gaze(bgr, iris_centre, np.mean(gaze_history, axis=0),
                                            length=120.0, thickness=1)
                    else:
                        gaze_history.clear()

                    if can_use_eyelid:
                        cv.polylines(
                            bgr, [np.round(eyelid_landmarks).astype(np.int32).reshape(-1, 1, 2)],
                            isClosed=True, color=(255, 255, 0), thickness=1, lineType=cv.LINE_AA,
                        )

                    if can_use_iris:
                        cv.polylines(
                            bgr, [np.round(iris_landmarks).astype(np.int32).reshape(-1, 1, 2)],
                            isClosed=True, color=(0, 255, 255), thickness=1, lineType=cv.LINE_AA,
                        )
                        cv.drawMarker(
                            bgr, tuple(np.round(iris_centre).astype(np.int32)),
                            color=(0, 255, 255), markerType=cv.MARKER_CROSS, markerSize=4,
                            thickness=1, line_type=cv.LINE_AA,
                        )

                    dtime = 1e3*(time.time() - start_time)
                    if 'visualization' not in frame['time']:
                        frame['time']['visualization'] = dtime
                    else:
                        frame['time']['visualization'] += dtime

                    def _dtime(before_id, after_id):
                        return int(1e3 * (frame['time'][after_id] - frame['time'][before_id]))

                    def _dstr(title, before_id, after_id):
                        return '%s: %dms' % (title, _dtime(before_id, after_id))

                    if eye_index == len(frame['eyes']) - 1:
                        # Calculate timings
                        frame['time']['after_visualization'] = time.time()
                        fps = int(np.round(1.0 / (time.time() - last_frame_time)))
                        fps_history.append(fps)
                        if len(fps_history) > 60:
                            fps_history = fps_history[-60:]
                        fps_str = '%d FPS' % np.mean(fps_history)
                        last_frame_time = time.time()
                        fh, fw, _ = bgr.shape
                        cv.putText(bgr, fps_str, org=(fw - 110, fh - 20),
                                   fontFace=cv.FONT_HERSHEY_DUPLEX, fontScale=0.8,
                                   color=(0, 0, 0), thickness=1, lineType=cv.LINE_AA)
                        cv.putText(bgr, fps_str, org=(fw - 111, fh - 21),
                                   fontFace=cv.FONT_HERSHEY_DUPLEX, fontScale=0.79,
                                   color=(255, 255, 255), thickness=1, lineType=cv.LINE_AA)
                        if not args.headless:
                            cv.imshow('vis', bgr)
                        last_frame_index = frame_index

                        # Record frame?
                        if args.record_video:
                            video_out_queue.put_nowait(frame_index)

                        # Quit?
                        if cv.waitKey(1) & 0xFF == ord('q'):
                            return

                        # Print timings
                        if frame_index % 60 == 0:
                            latency = _dtime('before_frame_read', 'after_visualization')
                            processing = _dtime('after_frame_read', 'after_visualization')
                            timing_string = ', '.join([
                                _dstr('read', 'before_frame_read', 'after_frame_read'),
                                _dstr('preproc', 'after_frame_read', 'after_preprocessing'),
                                'infer: %dms' % int(frame['time']['inference']),
                                'vis: %dms' % int(frame['time']['visualization']),
                                'proc: %dms' % processing,
                                'latency: %dms' % latency,
                            ])
                            print('%08d [%s] %s' % (frame_index, fps_str, timing_string))

        visualize_thread = threading.Thread(target=_visualize_output, name='visualization')
        visualize_thread.daemon = True
        visualize_thread.start()

        # Do inference forever
        infer = model.inference_generator()
        while True:
            output = next(infer)
            for frame_index in np.unique(output['frame_index']):
                if frame_index not in data_source._frames:
                    continue
                frame = data_source._frames[frame_index]
                if 'inference' in frame['time']:
                    frame['time']['inference'] += output['inference_time']
                else:
                    frame['time']['inference'] = output['inference_time']
            inferred_stuff_queue.put_nowait(output)

            if not visualize_thread.isAlive():
                break

            if not data_source._open:
                break

        # Close video recording
        if record_video and video_out is not None:
            video_out_should_stop = True
            video_out_queue.put_nowait(None)
            with video_out_done:
                video_out_done.wait()

In [ ]:
from typing import Dict

import numpy as np
import scipy
import tensorflow as tf

##from core import BaseDataSource, BaseModel


def _tf_mse(x, y):
    """Tensorflow call for mean-squared error."""
    return tf.reduce_mean(tf.squared_difference(x, y))


class ELG(BaseModel):
    """ELG architecture as introduced in [Park et al. ETRA'18]."""

    def __init__(self, tensorflow_session=None, first_layer_stride=1,
                 num_modules=2, num_feature_maps=32, **kwargs):
        """Specify ELG-specific parameters."""
        self._hg_first_layer_stride = first_layer_stride
        self._hg_num_modules = num_modules
        self._hg_num_feature_maps= num_feature_maps

        # Call parent class constructor
        super().__init__(tensorflow_session, **kwargs)

    _hg_first_layer_stride = 1
    _hg_num_modules = 2
    _hg_num_feature_maps = 32
    _hg_num_landmarks = 18
    _hg_num_residual_blocks = 1

    @property
    def identifier(self):
        """Identifier for model based on data sources and parameters."""
        first_data_source = next(iter(self._train_data.values()))
        input_tensors = first_data_source.output_tensors
        if self._data_format == 'NHWC':
            _, eh, ew, _ = input_tensors['eye'].shape.as_list()
        else:
            _, _, eh, ew = input_tensors['eye'].shape.as_list()
        return 'ELG_i%dx%d_f%dx%d_n%d_m%d' % (
            ew, eh,
            int(ew / self._hg_first_layer_stride),
            int(eh / self._hg_first_layer_stride),
            self._hg_num_feature_maps, self._hg_num_modules,
        )

    def train_loop_pre(self, current_step):
        """Run this at beginning of training loop."""
        # Set difficulty of training data
        data_source = next(iter(self._train_data.values()))
        data_source.set_difficulty(min((1. / 1e6) * current_step, 1.))

    def build_model(self, data_sources: Dict[str, BaseDataSource], mode: str):
        """Build model."""
        print("called ELG build model")
        data_source = next(iter(data_sources.values()))
        input_tensors = data_source.output_tensors
        x = input_tensors['eye']
        y1 = input_tensors['heatmaps'] if 'heatmaps' in input_tensors else None
        y2 = input_tensors['landmarks'] if 'landmarks' in input_tensors else None
        y3 = input_tensors['radius'] if 'radius' in input_tensors else None

        with tf.variable_scope('input_data'):
            self.summary.feature_maps('eyes', x, data_format=self._data_format_longer)
            if y1 is not None:
                self.summary.feature_maps('hmaps_true', y1, data_format=self._data_format_longer)

        outputs = {}
        loss_terms = {}
        metrics = {}

        with tf.variable_scope('hourglass'):
            # TODO: Find better way to specify no. landmarks
            if y1 is not None:
                if self._data_format == 'NCHW':
                    self._hg_num_landmarks = y1.shape.as_list()[1]
                if self._data_format == 'NHWC':
                    self._hg_num_landmarks = y1.shape.as_list()[3]
            else:
                self._hg_num_landmarks = 18
            assert self._hg_num_landmarks == 18

            # Prepare for Hourglass by downscaling via conv
            with tf.variable_scope('pre'):
                n = self._hg_num_feature_maps
                x = self._apply_conv(x, num_features=n, kernel_size=7,
                                     stride=self._hg_first_layer_stride)
                x = tf.nn.relu(self._apply_bn(x))
                x = self._build_residual_block(x, n, 2*n, name='res1')
                x = self._build_residual_block(x, 2*n, n, name='res2')

            # Hourglass blocks
            x_prev = x
            for i in range(self._hg_num_modules):
                with tf.variable_scope('hg_%d' % (i + 1)):
                    x = self._build_hourglass(x, steps_to_go=4, num_features=self._hg_num_feature_maps)
                    x, h = self._build_hourglass_after(
                        x_prev, x, do_merge=(i < (self._hg_num_modules - 1)),
                    )
                    self.summary.feature_maps('hmap%d' % i, h, data_format=self._data_format_longer)
                    if y1 is not None:
                        metrics['heatmap%d_mse' % (i + 1)] = _tf_mse(h, y1)
                    x_prev = x
            if y1 is not None:
                loss_terms['heatmaps_mse'] = tf.reduce_mean([
                    metrics['heatmap%d_mse' % (i + 1)] for i in range(self._hg_num_modules)
                ])
            x = h
            outputs['heatmaps'] = x

        # Soft-argmax
        x = self._calculate_landmarks(x)
        with tf.variable_scope('upscale'):
            # Upscale since heatmaps are half-scale of original image
            x *= self._hg_first_layer_stride
            if y2 is not None:
                metrics['landmarks_mse'] = _tf_mse(x, y2)
            outputs['landmarks'] = x

        # Fully-connected layers for radius regression
        with tf.variable_scope('radius'):
            x = tf.contrib.layers.flatten(tf.transpose(x, perm=[0, 2, 1]))
            for i in range(3):
                with tf.variable_scope('fc%d' % (i + 1)):
                    x = tf.nn.relu(self._apply_bn(self._apply_fc(x, 100)))
            with tf.variable_scope('out'):
                x = self._apply_fc(x, 1)
            outputs['radius'] = x
            if y3 is not None:
                metrics['radius_mse'] = _tf_mse(tf.reshape(x, [-1]), y3)
                loss_terms['radius_mse'] = 1e-7 * metrics['radius_mse']
            self.summary.histogram('radius', x)

        # Define outputs
        return outputs, loss_terms, metrics

    def _apply_conv(self, tensor, num_features, kernel_size=3, stride=1):
        return tf.layers.conv2d(
            tensor,
            num_features,
            kernel_size=kernel_size,
            strides=stride,
            padding='SAME',
            kernel_initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.01),
            kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-4),
            bias_initializer=tf.zeros_initializer(),
            data_format=self._data_format_longer,
            name='conv',
        )

    def _apply_fc(self, tensor, num_outputs):
        return tf.layers.dense(
            tensor,
            num_outputs,
            use_bias=True,
            kernel_initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.01),
            kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-4),
            bias_initializer=tf.zeros_initializer(),
            name='fc',
        )

    def _apply_pool(self, tensor, kernel_size=3, stride=2):
        tensor = tf.layers.max_pooling2d(
            tensor,
            pool_size=kernel_size,
            strides=stride,
            padding='SAME',
            data_format=self._data_format_longer,
            name='pool',
        )
        return tensor

    def _apply_bn(self, tensor):
        return tf.contrib.layers.batch_norm(
            tensor,
            scale=True,
            center=True,
            is_training=self.use_batch_statistics,
            trainable=True,
            data_format=self._data_format,
            updates_collections=None,
        )

    def _build_residual_block(self, x, num_in, num_out, name='res_block'):
        with tf.variable_scope(name):
            half_num_out = max(int(num_out/2), 1)
            c = x
            with tf.variable_scope('conv1'):
                c = tf.nn.relu(self._apply_bn(c))
                c = self._apply_conv(c, num_features=half_num_out, kernel_size=1, stride=1)
            with tf.variable_scope('conv2'):
                c = tf.nn.relu(self._apply_bn(c))
                c = self._apply_conv(c, num_features=half_num_out, kernel_size=3, stride=1)
            with tf.variable_scope('conv3'):
                c = tf.nn.relu(self._apply_bn(c))
                c = self._apply_conv(c, num_features=num_out, kernel_size=1, stride=1)
            with tf.variable_scope('skip'):
                if num_in == num_out:
                    s = tf.identity(x)
                else:
                    s = self._apply_conv(x, num_features=num_out, kernel_size=1, stride=1)
            x = c + s
        return x

    def _build_hourglass(self, x, steps_to_go, num_features, depth=1):
        with tf.variable_scope('depth%d' % depth):
            # Upper branch
            up1 = x
            for i in range(self._hg_num_residual_blocks):
                up1 = self._build_residual_block(up1, num_features, num_features,
                                                 name='up1_%d' % (i + 1))
            # Lower branch
            low1 = self._apply_pool(x, kernel_size=2, stride=2)
            for i in range(self._hg_num_residual_blocks):
                low1 = self._build_residual_block(low1, num_features, num_features,
                                                  name='low1_%d' % (i + 1))
            # Recursive
            low2 = None
            if steps_to_go > 1:
                low2 = self._build_hourglass(low1, steps_to_go - 1, num_features, depth=depth+1)
            else:
                low2 = low1
                for i in range(self._hg_num_residual_blocks):
                    low2 = self._build_residual_block(low2, num_features, num_features,
                                                      name='low2_%d' % (i + 1))
            # Additional residual blocks
            low3 = low2
            for i in range(self._hg_num_residual_blocks):
                low3 = self._build_residual_block(low3, num_features, num_features,
                                                  name='low3_%d' % (i + 1))
            # Upsample
            if self._data_format == 'NCHW':  # convert to NHWC
                low3 = tf.transpose(low3, (0, 2, 3, 1))
            up2 = tf.image.resize_bilinear(
                    low3,
                    up1.shape[1:3] if self._data_format == 'NHWC' else up1.shape[2:4],
                    align_corners=True,
                  )
            if self._data_format == 'NCHW':  # convert back from NHWC
                up2 = tf.transpose(up2, (0, 3, 1, 2))

        return up1 + up2

    def _build_hourglass_after(self, x_prev, x_now, do_merge=True):
        with tf.variable_scope('after'):
            for j in range(self._hg_num_residual_blocks):
                x_now = self._build_residual_block(x_now, self._hg_num_feature_maps,
                                                   self._hg_num_feature_maps,
                                                   name='after_hg_%d' % (j + 1))
            x_now = self._apply_conv(x_now, self._hg_num_feature_maps, kernel_size=1, stride=1)
            x_now = self._apply_bn(x_now)
            x_now = tf.nn.relu(x_now)

            with tf.variable_scope('hmap'):
                h = self._apply_conv(x_now, self._hg_num_landmarks, kernel_size=1, stride=1)

        x_next = x_now
        if do_merge:
            with tf.variable_scope('merge'):
                with tf.variable_scope('h'):
                    x_hmaps = self._apply_conv(h, self._hg_num_feature_maps, kernel_size=1, stride=1)
                with tf.variable_scope('x'):
                    x_now = self._apply_conv(x_now, self._hg_num_feature_maps, kernel_size=1, stride=1)
                x_next += x_prev + x_hmaps
        return x_next, h

    _softargmax_coords = None

    def _calculate_landmarks(self, x):
        """Estimate landmark location from heatmaps."""
        with tf.variable_scope('argsoftmax'):
            if self._data_format == 'NHWC':
                _, h, w, _ = x.shape.as_list()
            else:
                _, _, h, w = x.shape.as_list()
            if self._softargmax_coords is None:
                # Assume normalized coordinate [0, 1] for numeric stability
                ref_xs, ref_ys = np.meshgrid(np.linspace(0, 1.0, num=w, endpoint=True),
                                             np.linspace(0, 1.0, num=h, endpoint=True),
                                             indexing='xy')
                ref_xs = np.reshape(ref_xs, [-1, h*w])
                ref_ys = np.reshape(ref_ys, [-1, h*w])
                self._softargmax_coords = (
                    tf.constant(ref_xs, dtype=tf.float32),
                    tf.constant(ref_ys, dtype=tf.float32),
                )
            ref_xs, ref_ys = self._softargmax_coords

            # Assuming N x 18 x 45 x 75 (NCHW)
            beta = 1e2
            if self._data_format == 'NHWC':
                x = tf.transpose(x, (0, 3, 1, 2))
            x = tf.reshape(x, [-1, self._hg_num_landmarks, h*w])
            x = tf.nn.softmax(beta * x, axis=-1)
            lmrk_xs = tf.reduce_sum(ref_xs * x, axis=[2])
            lmrk_ys = tf.reduce_sum(ref_ys * x, axis=[2])

            # Return to actual coordinates ranges
            return tf.stack([
                lmrk_xs * (w - 1.0) + 0.5,
                lmrk_ys * (h - 1.0) + 0.5,
            ], axis=2)  # N x 18 x 2


def estimate_gaze_from_landmarks(iris_landmarks, iris_centre, eyeball_centre, eyeball_radius,
                                 initial_gaze=None):
    """Given iris edge landmarks and other coordinates, estimate gaze direction.
    More correctly stated, estimate gaze from iris edge landmark coordinates, iris centre
    coordinates, eyeball centre coordinates, and eyeball radius in pixels.
    """
    e_x0, e_y0 = eyeball_centre
    i_x0, i_y0 = iris_centre

    if initial_gaze is not None:
        theta, phi = initial_gaze
        # theta = -theta
    else:
        theta = np.arcsin(np.clip((i_y0 - e_y0) / eyeball_radius, -1.0, 1.0))
        phi = np.arcsin(np.clip((i_x0 - e_x0) / (eyeball_radius * -np.cos(theta)), -1.0, 1.0))

    delta = 0.1 * np.pi
    if iris_landmarks[0, 0] < iris_landmarks[4, 0]:  # flipped
        alphas = np.flip(np.arange(0.0, 2.0 * np.pi, step=np.pi/4.0), axis=0)
    else:
        alphas = np.arange(-np.pi, np.pi, step=np.pi/4.0) + np.pi/4.0
    sin_alphas = np.sin(alphas)
    cos_alphas = np.cos(alphas)

    def gaze_fit_loss_func(inputs):
        theta, phi, delta, phase = inputs
        sin_phase = np.sin(phase)
        cos_phase = np.cos(phase)
        # sin_alphas_shifted = np.sin(alphas + phase)
        sin_alphas_shifted = sin_alphas * cos_phase + cos_alphas * sin_phase
        # cos_alphas_shifted = np.cos(alphas + phase)
        cos_alphas_shifted = cos_alphas * cos_phase - sin_alphas * sin_phase

        sin_theta = np.sin(theta)
        cos_theta = np.cos(theta)
        sin_phi = np.sin(phi)
        cos_phi = np.cos(phi)
        sin_delta_sin = np.sin(delta * sin_alphas_shifted)
        sin_delta_cos = np.sin(delta * cos_alphas_shifted)
        cos_delta_sin = np.cos(delta * sin_alphas_shifted)
        cos_delta_cos = np.cos(delta * cos_alphas_shifted)
        # x = -np.cos(theta + delta * sin_alphas_shifted)
        x1 = -cos_theta * cos_delta_sin + sin_theta * sin_delta_sin
        # x *= np.sin(phi + delta * cos_alphas_shifted)
        x2 = sin_phi * cos_delta_cos + cos_phi * sin_delta_cos
        x = x1 * x2
        # y = np.sin(theta + delta * sin_alphas_shifted)
        y1 = sin_theta * cos_delta_sin
        y2 = cos_theta * sin_delta_sin
        y = y1 + y2

        ix = e_x0 + eyeball_radius * x
        iy = e_y0 + eyeball_radius * y
        dx = ix - iris_landmarks[:, 0]
        dy = iy - iris_landmarks[:, 1]
        out = np.mean(dx ** 2 + dy ** 2)

        # In addition, match estimated and actual iris centre
        iris_dx = e_x0 + eyeball_radius * -cos_theta * sin_phi - i_x0
        iris_dy = e_y0 + eyeball_radius * sin_theta - i_y0
        out += iris_dx ** 2 + iris_dy ** 2

        # sin_alphas_shifted = sin_alphas * cos_phase + cos_alphas * sin_phase
        # cos_alphas_shifted = cos_alphas * cos_phase - sin_alphas * sin_phase
        dsin_alphas_shifted_dphase = -sin_alphas * sin_phase + cos_alphas * cos_phase
        dcos_alphas_shifted_dphase = -cos_alphas * sin_phase - sin_alphas * cos_phase

        # sin_delta_sin = np.sin(delta * sin_alphas_shifted)
        # sin_delta_cos = np.sin(delta * cos_alphas_shifted)
        # cos_delta_sin = np.cos(delta * sin_alphas_shifted)
        # cos_delta_cos = np.cos(delta * cos_alphas_shifted)
        dsin_delta_sin_ddelta = cos_delta_sin * sin_alphas_shifted
        dsin_delta_cos_ddelta = cos_delta_cos * cos_alphas_shifted
        dcos_delta_sin_ddelta = -sin_delta_sin * sin_alphas_shifted
        dcos_delta_cos_ddelta = -sin_delta_cos * cos_alphas_shifted
        dsin_delta_sin_dphase = cos_delta_sin * delta * dsin_alphas_shifted_dphase
        dsin_delta_cos_dphase = cos_delta_cos * delta * dcos_alphas_shifted_dphase
        dcos_delta_sin_dphase = -sin_delta_sin * delta * dsin_alphas_shifted_dphase
        dcos_delta_cos_dphase = -sin_delta_cos * delta * dcos_alphas_shifted_dphase

        # x1 = -cos_theta * cos_delta_sin + sin_theta * sin_delta_sin
        # x2 = sin_phi * cos_delta_cos + cos_phi * sin_delta_cos
        dx1_dtheta = sin_theta * cos_delta_sin + cos_theta * sin_delta_sin
        dx2_dtheta = 0.0
        dx1_dphi = 0.0
        dx2_dphi = cos_phi * cos_delta_cos - sin_phi * sin_delta_cos
        dx1_ddelta = -cos_theta * dcos_delta_sin_ddelta + sin_theta * dsin_delta_sin_ddelta
        dx2_ddelta = sin_phi * dcos_delta_cos_ddelta + cos_phi * dsin_delta_cos_ddelta
        dx1_dphase = -cos_theta * dcos_delta_sin_dphase + sin_theta * dsin_delta_sin_dphase
        dx2_dphase = sin_phi * dcos_delta_cos_dphase + cos_phi * dsin_delta_cos_dphase

        # y1 = sin_theta * cos_delta_sin
        # y2 = cos_theta * sin_delta_sin
        dy1_dtheta = cos_theta * cos_delta_sin
        dy2_dtheta = -sin_theta * sin_delta_sin
        dy1_dphi = 0.0
        dy2_dphi = 0.0
        dy1_ddelta = sin_theta * dcos_delta_sin_ddelta
        dy2_ddelta = cos_theta * dsin_delta_sin_ddelta
        dy1_dphase = sin_theta * dcos_delta_sin_dphase
        dy2_dphase = cos_theta * dsin_delta_sin_dphase

        # x = x1 * x2
        # y = y1 + y2
        dx_dtheta = dx1_dtheta * x2 + x1 * dx2_dtheta
        dx_dphi = dx1_dphi * x2 + x1 * dx2_dphi
        dx_ddelta = dx1_ddelta * x2 + x1 * dx2_ddelta
        dx_dphase = dx1_dphase * x2 + x1 * dx2_dphase
        dy_dtheta = dy1_dtheta + dy2_dtheta
        dy_dphi = dy1_dphi + dy2_dphi
        dy_ddelta = dy1_ddelta + dy2_ddelta
        dy_dphase = dy1_dphase + dy2_dphase

        # ix = w_2 + eyeball_radius * x
        # iy = h_2 + eyeball_radius * y
        dix_dtheta = eyeball_radius * dx_dtheta
        dix_dphi = eyeball_radius * dx_dphi
        dix_ddelta = eyeball_radius * dx_ddelta
        dix_dphase = eyeball_radius * dx_dphase
        diy_dtheta = eyeball_radius * dy_dtheta
        diy_dphi = eyeball_radius * dy_dphi
        diy_ddelta = eyeball_radius * dy_ddelta
        diy_dphase = eyeball_radius * dy_dphase

        # dx = ix - iris_landmarks[:, 0]
        # dy = iy - iris_landmarks[:, 1]
        ddx_dtheta = dix_dtheta
        ddx_dphi = dix_dphi
        ddx_ddelta = dix_ddelta
        ddx_dphase = dix_dphase
        ddy_dtheta = diy_dtheta
        ddy_dphi = diy_dphi
        ddy_ddelta = diy_ddelta
        ddy_dphase = diy_dphase

        # out = dx ** 2 + dy ** 2
        dout_dtheta = np.mean(2 * (dx * ddx_dtheta + dy * ddy_dtheta))
        dout_dphi = np.mean(2 * (dx * ddx_dphi + dy * ddy_dphi))
        dout_ddelta = np.mean(2 * (dx * ddx_ddelta + dy * ddy_ddelta))
        dout_dphase = np.mean(2 * (dx * ddx_dphase + dy * ddy_dphase))

        # iris_dx = e_x0 + eyeball_radius * -cos_theta * sin_phi - i_x0
        # iris_dy = e_y0 + eyeball_radius * sin_theta - i_y0
        # out += iris_dx ** 2 + iris_dy ** 2
        dout_dtheta += 2 * eyeball_radius * (sin_theta * sin_phi * iris_dx + cos_theta * iris_dy)
        dout_dphi += 2 * eyeball_radius * (-cos_theta * cos_phi * iris_dx)

        return out, np.array([dout_dtheta, dout_dphi, dout_ddelta, dout_dphase])

    phase = 0.02
    result = scipy.optimize.minimize(gaze_fit_loss_func, x0=[theta, phi, delta, phase],
                                     bounds=(
                                         (-0.4*np.pi, 0.4*np.pi),
                                         (-0.4*np.pi, 0.4*np.pi),
                                         (0.01*np.pi, 0.5*np.pi),
                                         (-np.pi, np.pi),
                                     ),
                                     jac=True,
                                     tol=1e-6,
                                     method='TNC',
                                     options={
                                         # 'disp': True,
                                         'gtol': 1e-6,
                                         'maxiter': 100,
                                    })
    if result.success:
        theta, phi, delta, phase = result.x

    return np.array([-theta, phi])

In [ ]:
"""Main script for training a model for gaze estimation."""
import argparse

import coloredlogs
import tensorflow as tf

def mainELG():

    # Set global log level
    """parser = argparse.ArgumentParser(description='Train a gaze estimation model.')
    parser.add_argument('-v', type=str, help='logging level', default='info',
                        choices=['debug', 'info', 'warning', 'error', 'critical'])
    args = parser.parse_args()"""
    coloredlogs.install(
        datefmt='%d/%m %H:%M',
        fmt='%(asctime)s %(levelname)s %(message)s',
        level="info".upper(),   ##args.v.upper(),
    )

    # Initialize Tensorflow session
    tf.logging.set_verbosity(tf.logging.ERROR)
    gpu_options = tf.GPUOptions(allow_growth=True)
    with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as session:

        # Declare some parameters
        batch_size = 32

        # Define some model-specific parameters
        elg_first_layer_stride = 1
        elg_num_modules = 3
        elg_num_feature_maps = 32

        # Define training data source
        #from datasources import UnityEyes
        unityeyes = UnityEyes(
            session,
            batch_size=batch_size,
            data_format='NCHW',
            unityeyes_path="/content/gdrive/My Drive/test-data-Eye-tracking/imgs/imgs",
            min_after_dequeue=1000,
            generate_heatmaps=True,
            shuffle=True,
            staging=True,
            eye_image_shape=(36, 60),
            heatmaps_scale=1.0 / elg_first_layer_stride,
        )
        unityeyes.set_augmentation_range('translation', 2.0, 10.0)
        unityeyes.set_augmentation_range('rotation', 1.0, 10.0)
        unityeyes.set_augmentation_range('intensity', 0.5, 20.0)
        unityeyes.set_augmentation_range('blur', 0.1, 1.0)
        unityeyes.set_augmentation_range('scale', 0.01, 0.1)
        unityeyes.set_augmentation_range('rescale', 1.0, 0.5)
        unityeyes.set_augmentation_range('num_line', 0.0, 2.0)
        unityeyes.set_augmentation_range('heatmap_sigma', 7.5, 2.5)

        # Define model
        #from models import ELG
        model = ELG(
            # Tensorflow session
            # Note: The same session must be used for the model and the data sources.
            session,

            # Model configuration parameters
            # first_layer_stride describes how much the input image is downsampled before producing
            #                    feature maps for eventual heatmaps regression
            # num_modules defines the number of hourglass modules, and thus the number of times repeated
            #             coarse-to-fine refinement is done.
            # num_feature_maps describes how many feature maps are refined over the entire network.
            first_layer_stride=elg_first_layer_stride,
            num_feature_maps=elg_num_feature_maps,
            num_modules=elg_num_modules,

            # The learning schedule describes in which order which part of the network should be
            # trained and with which learning rate.
            #
            # A standard network would have one entry (dict) in this argument where all model
            # parameters are optimized. To do this, you must specify which variables must be
            # optimized and this is done by specifying which prefixes to look for.
            # The prefixes are defined by using `tf.variable_scope`.
            #
            # The loss terms which can be specified depends on model specifications, specifically
            # the `loss_terms` output of `BaseModel::build_model`.
            learning_schedule=[
                {
                    'loss_terms_to_optimize': {
                        'heatmaps_mse': ['hourglass'],
                        'radius_mse': ['radius'],
                    },
                    'learning_rate': 1e-3,
                },
            ],

            # Data sources for training (and testing).
            train_data={'synthetic': unityeyes},
        )

        # Train this model for a set number of epochs
        model.train(
            num_epochs=100,
        )


In [ ]:
#!/usr/bin/env python3
"""Main script for gaze direction inference from webcam feed."""
import argparse
import os
import queue
import threading
import time

import coloredlogs
import cv2 as cv
import numpy as np
import tensorflow as tf

"""from datasources import Video, Webcam
from models import ELG
import util.gaze"""

##if __name__ == '__main__':
def test_elg():
    # Set global log level
    """parser = argparse.ArgumentParser(description='Demonstration of landmarks localization.')
    parser.add_argument('-v', type=str, help='logging level', default='info',
                        choices=['debug', 'info', 'warning', 'error', 'critical'])
    parser.add_argument('--from_video', type=str, help='Use this video path instead of webcam')
    parser.add_argument('--record_video', type=str, help='Output path of video of demonstration.')
    parser.add_argument('--fullscreen', action='store_true')
    parser.add_argument('--headless', action='store_true')

    parser.add_argument('--fps', type=int, default=60, help='Desired sampling rate of webcam')
    parser.add_argument('--camera_id', type=int, default=0, help='ID of webcam to use')"""

    ##Store local
    video_path = "/content/"
    video_name = "test7.mp4"
    from_video = video_path + video_name
    record_video = "/content/outputs/output.mp4"
    fullscreen = False
    headless = False
    fps = 60
    ##args = parser.parse_args()
    coloredlogs.install(
        datefmt='%d/%m %H:%M',
        fmt='%(asctime)s %(levelname)s %(message)s',
        level="debug".upper(),   ##args.v.upper(),
    )

    # Check if GPU is available
    from tensorflow.python.client import device_lib
    session_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
    gpu_available = False
    try:
        gpus = [d for d in device_lib.list_local_devices(config=session_config)
                if d.device_type == 'GPU']
        gpu_available = len(gpus) > 0
    except:
        pass

    # Initialize Tensorflow session
    tf.reset_default_graph()

    tf.logging.set_verbosity(tf.logging.INFO)
    with tf.Session(config=session_config) as session:

        # Declare some parameters
        batch_size = 2

        # Define webcam stream data source
        # Change data_format='NHWC' if not using CUDA
        if from_video:
            assert os.path.isfile(from_video)
            data_source = Video(from_video,
                                tensorflow_session=session, batch_size=batch_size,
                                data_format='NCHW' if gpu_available else 'NHWC',
                                eye_image_shape=(108, 180))
        else:
            data_source = Webcam(tensorflow_session=session, batch_size=batch_size,
                                 camera_id=args.camera_id, fps=args.fps,
                                 data_format='NCHW' if gpu_available else 'NHWC',
                                 eye_image_shape=(36, 60))

        # Define model
        if from_video:
            model = ELG(
                session, train_data={'videostream': data_source},
                first_layer_stride=3,
                num_modules=3,
                num_feature_maps=64,
                learning_schedule=[
                    {
                        'loss_terms_to_optimize': {'dummy': ['hourglass', 'radius']},
                    },
                ],
            )
        else:
            model = ELG(
                session, train_data={'videostream': data_source},
                first_layer_stride=1,
                num_modules=2,
                num_feature_maps=32,
                learning_schedule=[
                    {
                        'loss_terms_to_optimize': {'dummy': ['hourglass', 'radius']},
                    },
                ],
            )

        # Record output frames to file if requested
        if record_video:
            video_out = None
            video_out_queue = queue.Queue()
            video_out_should_stop = False
            video_out_done = threading.Condition()

            def _record_frame():
                
                print("Record thread started")
                nonlocal video_out
                last_frame_time = None
                out_fps = 30
                out_frame_interval = 1.0 / out_fps
                while not video_out_should_stop:
                    print("Creating frame for video_out")
                    frame_index = video_out_queue.get()
                    if frame_index is None:
                        print("No frame index from video_out_queue: breaks here ")
                        break
                    assert frame_index in data_source._frames
                    frame = data_source._frames[frame_index]['bgr']
                    h, w, _ = frame.shape
                    if video_out is None:
                        print("Writing to video file")
                        video_out = cv.VideoWriter(
                            record_video, cv.VideoWriter_fourcc(*"mp4v"),  ##'H264'),
                            out_fps, (w, h),
                        )
                    now_time = time.time()
                    if last_frame_time is not None:
                        time_diff = now_time - last_frame_time
                        while time_diff > 0.0:
                            #cv.imwrite('kang'+str(frameIndexx)+'.jpg',frame)
                            #frameIndexx += 1
                            video_out.write(frame)
                            time_diff -= out_frame_interval
                    last_frame_time = now_time
                video_out.release()
                with video_out_done:
                    video_out_done.notify_all()
            record_thread = threading.Thread(target=_record_frame, name='record')
            record_thread.daemon = True
            record_thread.start()

        # Begin visualization thread
        inferred_stuff_queue = queue.Queue()

        def _visualize_output():
            last_frame_index = 0
            last_frame_time = time.time()
            fps_history = []
            all_gaze_histories = []
            print("In visualise output")
            """if fullscreen:
                cv.namedWindow('vis', cv.WND_PROP_FULLSCREEN)
                cv.setWindowProperty('vis', cv.WND_PROP_FULLSCREEN, cv.WINDOW_FULLSCREEN)"""

            while True:
                # If no output to visualize, show unannotated frame
                if inferred_stuff_queue.empty():
                    next_frame_index = last_frame_index + 1
                    if next_frame_index in data_source._frames:
                        next_frame = data_source._frames[next_frame_index]
                        if 'faces' in next_frame and len(next_frame['faces']) == 0:
                            """if not headless:
                                cv.imshow('vis', next_frame['bgr'])"""
                            if record_video:
                                print("Adding frame index to video_out_queue")
                                video_out_queue.put_nowait(next_frame_index)
                            last_frame_index = next_frame_index
                    if cv.waitKey(1) & 0xFF == ord('q'):
                        return
                    continue

                # Get output from neural network and visualize
                output = inferred_stuff_queue.get()
                bgr = None
                for j in range(batch_size):
                    frame_index = output['frame_index'][j]
                    if frame_index not in data_source._frames:
                        continue
                    frame = data_source._frames[frame_index]
                    
                    # Decide which landmarks are usable
                    heatmaps_amax = np.amax(output['heatmaps'][j, :].reshape(-1, 18), axis=0)
                    can_use_eye = np.all(heatmaps_amax > 0.7)
                    can_use_eyelid = np.all(heatmaps_amax[0:8] > 0.75)
                    can_use_iris = np.all(heatmaps_amax[8:16] > 0.8)

                    start_time = time.time()
                    eye_index = output['eye_index'][j]
                    bgr = frame['bgr']
                    #cv.imwrite('kang'+str(j)+'.jpg',frame)
                    #frameIndexx += 1
                    eye = frame['eyes'][eye_index]
                    
                    eye_image = eye['image']
                    eye_side = eye['side']
                    eye_landmarks = output['landmarks'][j, :]
                    eye_radius = output['radius'][j][0]
                    if eye_side == 'left':
                        eye_landmarks[:, 0] = eye_image.shape[1] - eye_landmarks[:, 0]
                        eye_image = np.fliplr(eye_image)

                    # Embed eye image and annotate for picture-in-picture
                    eye_upscale = 2
                    eye_image_raw = cv.cvtColor(cv.equalizeHist(eye_image), cv.COLOR_GRAY2BGR)
                    eye_image_raw = cv.resize(eye_image_raw, (0, 0), fx=eye_upscale, fy=eye_upscale)
                    eye_image_annotated = np.copy(eye_image_raw)
                    if can_use_eyelid:
                        cv.polylines(
                            eye_image_annotated,
                            [np.round(eye_upscale*eye_landmarks[0:8]).astype(np.int32)
                                                                     .reshape(-1, 1, 2)],
                            isClosed=True, color=(255, 255, 0), thickness=1, lineType=cv.LINE_AA, ##Yellow Line
                        )
                    if can_use_iris:
                        cv.polylines(
                            eye_image_annotated,
                            [np.round(eye_upscale*eye_landmarks[8:16]).astype(np.int32)
                                                                      .reshape(-1, 1, 2)],
                            isClosed=True, color=(0, 255, 255), thickness=1, lineType=cv.LINE_AA, ##Blue Line
                        )
                        cv.drawMarker(
                            eye_image_annotated,
                            tuple(np.round(eye_upscale*eye_landmarks[16, :]).astype(np.int32)),
                            color=(0, 255, 255), markerType=cv.MARKER_CROSS, markerSize=4, ##Blue CROSS
                            thickness=1, line_type=cv.LINE_AA,
                        )
                    face_index = int(eye_index / 2)
                    eh, ew, _ = eye_image_raw.shape
                    v0 = face_index * 2 * eh
                    v1 = v0 + eh
                    v2 = v1 + eh
                    u0 = 0 if eye_side == 'left' else ew
                    u1 = u0 + ew
                    bgr[v0:v1, u0:u1] = eye_image_raw
                    bgr[v1:v2, u0:u1] = eye_image_annotated

                    # Visualize preprocessing results
                    frame_landmarks = (frame['smoothed_landmarks']
                                       if 'smoothed_landmarks' in frame
                                       else frame['landmarks'])
                    for f, face in enumerate(frame['faces']):
                        for landmark in frame_landmarks[f][:-1]:
                            cv.drawMarker(bgr, tuple(np.round(landmark).astype(np.int32)),
                                          color=(0, 0, 255), markerType=cv.MARKER_STAR,  ##BLUE STAR
                                          markerSize=2, thickness=1, line_type=cv.LINE_AA)
                        cv.rectangle(
                            bgr, tuple(np.round(face[:2]).astype(np.int32)),
                            tuple(np.round(np.add(face[:2], face[2:])).astype(np.int32)),
                            color=(0, 255, 255), thickness=1, lineType=cv.LINE_AA,
                        )

                    # Transform predictions
                    eye_landmarks = np.concatenate([eye_landmarks,
                                                    [[eye_landmarks[-1, 0] + eye_radius,
                                                      eye_landmarks[-1, 1]]]])
                    eye_landmarks = np.asmatrix(np.pad(eye_landmarks, ((0, 0), (0, 1)),
                                                       'constant', constant_values=1.0))
                    eye_landmarks = (eye_landmarks *
                                     eye['inv_landmarks_transform_mat'].T)[:, :2]
                    eye_landmarks = np.asarray(eye_landmarks)
                    eyelid_landmarks = eye_landmarks[0:8, :]
                    iris_landmarks = eye_landmarks[8:16, :]
                    iris_centre = eye_landmarks[16, :]
                    eyeball_centre = eye_landmarks[17, :]
                    eyeball_radius = np.linalg.norm(eye_landmarks[18, :] -
                                                    eye_landmarks[17, :])

                    # Smooth and visualize gaze direction
                    num_total_eyes_in_frame = len(frame['eyes'])
                    if len(all_gaze_histories) != num_total_eyes_in_frame:
                        all_gaze_histories = [list() for _ in range(num_total_eyes_in_frame)]
                    gaze_history = all_gaze_histories[eye_index]
                    if can_use_eye:
                        # Visualize landmarks
                        cv.drawMarker(  # Eyeball centre
                            bgr, tuple(np.round(eyeball_centre).astype(np.int32)),
                            color=(0, 255, 0), markerType=cv.MARKER_CROSS, markerSize=4, ##GREEN CENTRE of eyeball
                            thickness=1, line_type=cv.LINE_AA,
                        )
                        # cv.circle(  # Eyeball outline
                        #     bgr, tuple(np.round(eyeball_centre).astype(np.int32)),
                        #     int(np.round(eyeball_radius)), color=(0, 255, 0),
                        #     thickness=1, lineType=cv.LINE_AA,
                        # )

                        # Draw "gaze"
                        # from models.elg import estimate_gaze_from_landmarks
                        # current_gaze = estimate_gaze_from_landmarks(
                        #     iris_landmarks, iris_centre, eyeball_centre, eyeball_radius)
                        i_x0, i_y0 = iris_centre
                        e_x0, e_y0 = eyeball_centre
                        theta = -np.arcsin(np.clip((i_y0 - e_y0) / eyeball_radius, -1.0, 1.0))
                        phi = np.arcsin(np.clip((i_x0 - e_x0) / (eyeball_radius * -np.cos(theta)),
                                                -1.0, 1.0))
                        current_gaze = np.array([theta, phi])
                        gaze_history.append(current_gaze)
                        gaze_history_max_len = 10
                        if len(gaze_history) > gaze_history_max_len:
                            gaze_history = gaze_history[-gaze_history_max_len:]
                        draw_gaze(bgr, iris_centre, np.mean(gaze_history, axis=0),
                                            length=120.0, thickness=1)
                    else:
                        gaze_history.clear()

                    if can_use_eyelid:
                        cv.polylines(
                            bgr, [np.round(eyelid_landmarks).astype(np.int32).reshape(-1, 1, 2)],
                            isClosed=True, color=(255, 255, 0), thickness=1, lineType=cv.LINE_AA,
                        )

                    if can_use_iris:
                        cv.polylines(
                            bgr, [np.round(iris_landmarks).astype(np.int32).reshape(-1, 1, 2)],
                            isClosed=True, color=(0, 255, 255), thickness=1, lineType=cv.LINE_AA,  ##BLUE LINE ON EYE white
                        )
                        cv.drawMarker(
                            bgr, tuple(np.round(iris_centre).astype(np.int32)),
                            color=(0, 255, 255), markerType=cv.MARKER_CROSS, markerSize=4,
                            thickness=1, line_type=cv.LINE_AA,
                        )

                    dtime = 1e3*(time.time() - start_time)
                    if 'visualization' not in frame['time']:
                        frame['time']['visualization'] = dtime
                    else:
                        frame['time']['visualization'] += dtime

                    def _dtime(before_id, after_id):
                        return int(1e3 * (frame['time'][after_id] - frame['time'][before_id]))

                    def _dstr(title, before_id, after_id):
                        return '%s: %dms' % (title, _dtime(before_id, after_id))

                    if eye_index == len(frame['eyes']) - 1:
                        # Calculate timings
                        frame['time']['after_visualization'] = time.time()
                        fps = int(np.round(1.0 / (time.time() - last_frame_time)))
                        fps_history.append(fps)
                        if len(fps_history) > 60:
                            fps_history = fps_history[-60:]
                        fps_str = '%d FPS' % np.mean(fps_history)
                        last_frame_time = time.time()
                        fh, fw, _ = bgr.shape
                        cv.putText(bgr, fps_str, org=(fw - 110, fh - 20),
                                   fontFace=cv.FONT_HERSHEY_DUPLEX, fontScale=0.8,
                                   color=(0, 0, 0), thickness=1, lineType=cv.LINE_AA)
                        cv.putText(bgr, fps_str, org=(fw - 111, fh - 21),
                                   fontFace=cv.FONT_HERSHEY_DUPLEX, fontScale=0.79,
                                   color=(255, 255, 255), thickness=1, lineType=cv.LINE_AA)
                        """if not headless:
                            cv.imshow('vis', bgr)"""
                        last_frame_index = frame_index

                        # Record frame?
                        if record_video:
                            video_out_queue.put_nowait(frame_index)

                        # Quit?
                        if cv.waitKey(1) & 0xFF == ord('q'):
                            return

                        # Print timings
                        if frame_index % 60 == 0:
                            latency = _dtime('before_frame_read', 'after_visualization')
                            processing = _dtime('after_frame_read', 'after_visualization')
                            timing_string = ', '.join([
                                _dstr('read', 'before_frame_read', 'after_frame_read'),
                                _dstr('preproc', 'after_frame_read', 'after_preprocessing'),
                                'infer: %dms' % int(frame['time']['inference']),
                                'vis: %dms' % int(frame['time']['visualization']),
                                'proc: %dms' % processing,
                                'latency: %dms' % latency,
                            ])
                            print('%08d [%s] %s' % (frame_index, fps_str, timing_string))

        visualize_thread = threading.Thread(target=_visualize_output, name='visualization')
        visualize_thread.daemon = True
        visualize_thread.start()

        # Do inference forever
        infer = model.inference_generator()
        while True:
            output = next(infer)
            for frame_index in np.unique(output['frame_index']):
                if frame_index not in data_source._frames:
                    continue
                frame = data_source._frames[frame_index]
                """if 'inference' in frame['time']:
                    frame['time']['inference'] += output['inference_time']
                else:"""
                frame['time']['inference'] = output['inference_time']
            inferred_stuff_queue.put_nowait(output)

            if not visualize_thread.isAlive():
                break

            if not data_source._open:
                break

        # Close video recording
        if record_video and video_out is not None:
            print("Video_out is not Null here",video_out )
            video_out_should_stop = True
            video_out_queue.put_nowait(None)
            with video_out_done:
                video_out_done.wait()
            print("Video_out_done")

In [ ]:
"""UnityEyes data source for gaze estimation."""
import os
from threading import Lock

import cv2 as cv
import numpy as np
import tensorflow as tf
import ujson

"""from core import BaseDataSource
import util.gaze
import util.heatmap"""


class TestUnityEyes(BaseDataSource):
    """UnityEyes data loading class."""

    def __init__(self,
                 tensorflow_session: tf.Session,
                 batch_size: int,
                 unityeyes_path: str,
                 testing=False,
                 generate_heatmaps=False,
                 eye_image_shape=(36, 60),
                 heatmaps_scale=1.0,
                 **kwargs):
        """Create queues and threads to read and preprocess data."""
        self._short_name = 'TestUnityEyes'
        if testing:
            self._short_name += ':test'

        # Cache some parameters
        self._eye_image_shape = eye_image_shape
        self._heatmaps_scale = heatmaps_scale

        # Create global index over all specified keys
        self._images_path = unityeyes_path
        self._file_stems = sorted([p[:-5] for p in os.listdir(unityeyes_path)
                                   if p.endswith('.json')])
        self._num_entries = len(self._file_stems)
        print("Number of entries trained before",self._num_entries)
        self._num_entries = 3000
        print("Number of entries trained after",self._num_entries)
        self._mutex = Lock()
        self._current_index = 0

        # Define bounds for noise values for different augmentation types
        self._difficulty = 0.0
        self._augmentation_ranges = {  # (easy, hard)
            'translation': (2.0, 10.0),
            'rotation': (0.1, 2.0),
            'intensity': (0.5, 20.0),
            'blur': (0.1, 1.0),
            'scale': (0.01, 0.1),
            'rescale': (1.0, 0.2),
            'num_line': (0.0, 2.0),
            'heatmap_sigma': (5.0, 2.5),
        }
        self._generate_heatmaps = generate_heatmaps

        # Call parent class constructor
        super().__init__(tensorflow_session, batch_size=batch_size, testing=testing, **kwargs)

    @property
    def num_entries(self):
        """Number of entries in this data source."""
        return self._num_entries

    @property
    def short_name(self):
        """Short name specifying source UnityEyes."""
        return self._short_name

    def reset(self):
        """Reset index."""
        with self._mutex:
            super().reset()
            self._current_index = 0

    def entry_generator(self, yield_just_one=False):
        """Read entry from TestUnityEyes."""
        try:
            while range(1) if yield_just_one else True:
                with self._mutex:
                    if self._current_index >= self.num_entries:
                        if self.testing:
                            break
                        else:
                            self._current_index = 0
                    current_index = self._current_index
                    self._current_index += 1

                file_stem = self._file_stems[current_index]
                jpg_path = '%s/%s.jpg' % (self._images_path, file_stem)
                json_path = '%s/%s.json' % (self._images_path, file_stem)
                if not os.path.isfile(jpg_path) or not os.path.isfile(json_path):
                    continue
                with open(json_path, 'r') as f:
                    json_data = ujson.load(f)
                entry = {
                    'full_image': cv.imread(jpg_path, cv.IMREAD_GRAYSCALE),
                    'json_data': json_data,
                }
                assert entry['full_image'] is not None
                yield entry
        finally:
            # Execute any cleanup operations as necessary
            pass

    def set_difficulty(self, difficulty):
        """Set difficulty of training data."""
        assert isinstance(difficulty, float)
        assert 0.0 <= difficulty <= 1.0
        self._difficulty = difficulty

    def set_augmentation_range(self, augmentation_type, easy_value, hard_value):
        """Set 'range' for a known augmentation type."""
        assert isinstance(augmentation_type, str)
        assert augmentation_type in self._augmentation_ranges
        assert isinstance(easy_value, float) or isinstance(easy_value, int)
        assert isinstance(hard_value, float) or isinstance(hard_value, int)
        self._augmentation_ranges[augmentation_type] = (easy_value, hard_value)

    def preprocess_entry(self, entry):
        """Use annotations to segment eyes and calculate gaze direction."""
        full_image = entry['full_image']
        json_data = entry['json_data']
        del entry['full_image']
        del entry['json_data']

        ih, iw = full_image.shape
        iw_2, ih_2 = 0.5 * iw, 0.5 * ih
        oh, ow = self._eye_image_shape

        def process_coords(coords_list):
            coords = [eval(l) for l in coords_list]
            return np.array([(x, ih-y, z) for (x, y, z) in coords])
        interior_landmarks = process_coords(json_data['interior_margin_2d'])
        caruncle_landmarks = process_coords(json_data['caruncle_2d'])
        iris_landmarks = process_coords(json_data['iris_2d'])

        random_multipliers = []

        def value_from_type(augmentation_type):
            # Scale to be in range
            easy_value, hard_value = self._augmentation_ranges[augmentation_type]
            value = (hard_value - easy_value) * self._difficulty + easy_value
            value = (np.clip(value, easy_value, hard_value)
                     if easy_value < hard_value
                     else np.clip(value, hard_value, easy_value))
            return value

        def noisy_value_from_type(augmentation_type):
            # Get normal distributed random value
            if len(random_multipliers) == 0:
                random_multipliers.extend(
                        list(np.random.normal(size=(len(self._augmentation_ranges),))))
            return random_multipliers.pop() * value_from_type(augmentation_type)

        # Only select almost frontal images
        h_pitch, h_yaw, _ = eval(json_data['head_pose'])
        if h_pitch > 180.0:  # Need to correct pitch
            h_pitch -= 360.0
        h_yaw -= 180.0  # Need to correct yaw
        if abs(h_pitch) > 20 or abs(h_yaw) > 20:
            return None

        # Prepare to segment eye image
        left_corner = np.mean(caruncle_landmarks[:, :2], axis=0)
        right_corner = interior_landmarks[8, :2]
        eye_width = 1.5 * abs(left_corner[0] - right_corner[0])
        eye_middle = np.mean([np.amin(interior_landmarks[:, :2], axis=0),
                              np.amax(interior_landmarks[:, :2], axis=0)], axis=0)

        # Centre axes to eyeball centre
        translate_mat = np.asmatrix(np.eye(3))
        translate_mat[:2, 2] = [[-iw_2], [-ih_2]]

        # Rotate eye image if requested
        rotate_mat = np.asmatrix(np.eye(3))
        rotation_noise = noisy_value_from_type('rotation')
        if rotation_noise > 0:
            rotate_angle = np.radians(rotation_noise)
            cos_rotate = np.cos(rotate_angle)
            sin_rotate = np.sin(rotate_angle)
            rotate_mat[0, 0] = cos_rotate
            rotate_mat[0, 1] = -sin_rotate
            rotate_mat[1, 0] = sin_rotate
            rotate_mat[1, 1] = cos_rotate

        # Scale image to fit output dimensions (with a little bit of noise)
        scale_mat = np.asmatrix(np.eye(3))
        scale = 1. + noisy_value_from_type('scale')
        scale_inv = 1. / scale
        np.fill_diagonal(scale_mat, ow / eye_width * scale)
        original_eyeball_radius = 71.7593
        eyeball_radius = original_eyeball_radius * scale_mat[0, 0]  # See: https://goo.gl/ZnXgDE
        ##entry['radius'] = np.float32(eyeball_radius)

        # Re-centre eye image such that eye fits (based on determined `eye_middle`)
        recentre_mat = np.asmatrix(np.eye(3))
        recentre_mat[0, 2] = iw/2 - eye_middle[0] + 0.5 * eye_width * scale_inv
        recentre_mat[1, 2] = ih/2 - eye_middle[1] + 0.5 * oh / ow * eye_width * scale_inv
        recentre_mat[0, 2] += noisy_value_from_type('translation')  # x
        recentre_mat[1, 2] += noisy_value_from_type('translation')  # y

        # Apply transforms
        transform_mat = recentre_mat * scale_mat * rotate_mat * translate_mat
        eye = cv.warpAffine(full_image, transform_mat[:2, :3], (ow, oh))

        # Convert look vector to gaze direction in polar angles
        look_vec = np.array(eval(json_data['eye_details']['look_vec']))[:3]
        look_vec[0] = -look_vec[0]
        ##look_vector = 1 row 3 cols-> [ x y z ]
        original_gaze = vector_to_pitchyaw(look_vec.reshape((1, 3))).flatten() ##util.gaze.
        look_vec = rotate_mat * look_vec.reshape(3, 1)
        gaze = vector_to_pitchyaw(look_vec.reshape((1, 3))).flatten() ##util.gaze.
        if gaze[1] > 0.0:
            gaze[1] = np.pi - gaze[1]
        elif gaze[1] < 0.0:
            gaze[1] = -(np.pi + gaze[1])
        entry['gaze'] = gaze.astype(np.float32)

        # Draw line randomly
        num_line_noise = int(np.round(noisy_value_from_type('num_line')))
        if num_line_noise > 0:
            line_rand_nums = np.random.rand(5 * num_line_noise)
            for i in range(num_line_noise):
                j = 5 * i
                lx0, ly0 = int(ow * line_rand_nums[j]), oh
                lx1, ly1 = ow, int(oh * line_rand_nums[j + 1])
                direction = line_rand_nums[j + 2]
                if direction < 0.25:
                    lx1 = ly0 = 0
                elif direction < 0.5:
                    lx1 = 0
                elif direction < 0.75:
                    ly0 = 0
                line_colour = int(255 * line_rand_nums[j + 3])
                eye = cv.line(eye, (lx0, ly0), (lx1, ly1),
                              color=(line_colour, line_colour, line_colour),
                              thickness=max(1, int(6*line_rand_nums[j + 4])),
                              lineType=cv.LINE_AA)

        # Rescale image if required
        rescale_max = value_from_type('rescale')
        if rescale_max < 1.0:
            rescale_noise = np.random.uniform(low=rescale_max, high=1.0)
            interpolation = cv.INTER_CUBIC
            eye = cv.resize(eye, dsize=(0, 0), fx=rescale_noise, fy=rescale_noise,
                            interpolation=interpolation)
            eye = cv.equalizeHist(eye)
            eye = cv.resize(eye, dsize=(ow, oh), interpolation=interpolation)

        # Add rgb noise to eye image
        intensity_noise = int(value_from_type('intensity'))
        if intensity_noise > 0:
            eye = eye.astype(np.int16)
            eye += np.random.randint(low=-intensity_noise, high=intensity_noise,
                                     size=eye.shape, dtype=np.int16)
            cv.normalize(eye, eye, alpha=0, beta=255, norm_type=cv.NORM_MINMAX)
            eye = eye.astype(np.uint8)

        # Add blur to eye image
        blur_noise = noisy_value_from_type('blur')
        if blur_noise > 0:
            eye = cv.GaussianBlur(eye, (7, 7), 0.5 + np.abs(blur_noise))

        # Histogram equalization and preprocessing for NN
        eye = cv.equalizeHist(eye) ## increase contrast of the image
        eye = eye.astype(np.float32)
        eye *= 2.0 / 255.0
        eye -= 1.0
        eye = np.expand_dims(eye, -1 if self.data_format == 'NHWC' else 0)
        entry['eye'] = eye

        # Select and transform landmark coordinates
        iris_centre = np.asarray([
            iw_2 + original_eyeball_radius * -np.cos(original_gaze[0]) * np.sin(original_gaze[1]),
            ih_2 + original_eyeball_radius * -np.sin(original_gaze[0]),
        ])
        landmarks = np.concatenate([interior_landmarks[::2, :2],  # 8
                                    iris_landmarks[::4, :2],  # 8
                                    iris_centre.reshape((1, 2)),
                                    [[iw_2, ih_2]],  # Eyeball centre
                                    ])  # 18 in total
        landmarks = np.asmatrix(np.pad(landmarks, ((0, 0), (0, 1)), 'constant',
                                       constant_values=1))
        landmarks = np.asarray(landmarks * transform_mat.T)
        landmarks = landmarks[:, :2]  # We only need x, y
        entry['landmarks'] = landmarks.astype(np.float32)

        # Generate heatmaps if necessary
        """if self._generate_heatmaps:
            # Should be half-scale (compared to eye image). # util.heatmap.gaussian_2d
            entry['heatmaps'] = np.asarray([
                    gaussian_2d(
                    shape=(self._heatmaps_scale*oh, self._heatmaps_scale*ow),
                    centre=self._heatmaps_scale*landmark,
                    sigma=value_from_type('heatmap_sigma'),
                )
                for landmark in entry['landmarks']
            ]).astype(np.float32)
            if self.data_format == 'NHWC':
                entry['heatmaps'] = np.transpose(entry['heatmaps'], (1, 2, 0))"""

        return entry


In [ ]:
##test_elg()

import argparse
import os
import queue
import threading
import time

import coloredlogs
import cv2 as cv
import numpy as np
import tensorflow as tf

"""from datasources import Video, Webcam
from models import ELG
import util.gaze"""
mean_angular_errors = [] 
predicted = []
actual_gaze = []
def testUnity_elg():
    
    
    coloredlogs.install(
        datefmt='%d/%m %H:%M',
        fmt='%(asctime)s %(levelname)s %(message)s',
        level="debug".upper(),   
    )

    # Check if GPU is available
    from tensorflow.python.client import device_lib
    session_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
    gpu_available = False
    try:
        gpus = [d for d in device_lib.list_local_devices(config=session_config)
                if d.device_type == 'GPU']
        gpu_available = len(gpus) > 0
    except:
        pass
    

    # Initialize Tensorflow session
    tf.reset_default_graph()

    tf.logging.set_verbosity(tf.logging.INFO)
    with tf.Session(config=session_config) as session:

        # Declare some parameters
        batch_size = 2

        
        data_source = TestUnityEyes(
            tensorflow_session=session,
            batch_size=batch_size,
            data_format='NCHW' if gpu_available else 'NHWC',
            unityeyes_path=image_dir, #"/content/gdrive/My Drive/test-data-Eye-tracking/imgs/imgs",
            #min_after_dequeue=1000,
            generate_heatmaps=False, #True,
            #shuffle=True,
            #staging=True,
            eye_image_shape=(36, 60)  #,
            #heatmaps_scale=1.0 / elg_first_layer_stride,
        )
        # Define model
        model = ELG(
                session, train_data={'testUnity': data_source},
                first_layer_stride=1,
                num_modules=2,
                num_feature_maps=32,
                learning_schedule=[
                    {
                        'loss_terms_to_optimize': {'dummy': ['hourglass', 'radius']},
                    },
                ],
            )
            
        print("model complete")
        infer = model.inference_generator()
        print("inference complete", infer)
        i = 0
        while True:
          output = next(infer)
          ##print("output",output)
          #print("output['landmarks']",output['landmarks'], output['landmarks'].shape)
          eye_landmarks = output['landmarks'][1, :]
          eye_radius = output['radius'][1][0]

          eye_landmarks = np.concatenate([eye_landmarks,
                                          [[eye_landmarks[-1, 0] + eye_radius,
                                            eye_landmarks[-1, 1]]]])
          eye_landmarks = np.asmatrix(np.pad(eye_landmarks, ((0, 0), (0, 1)),
                                              'constant', constant_values=1.0))
          """eye_landmarks = (eye_landmarks *
                            eye['inv_landmarks_transform_mat'].T)[:, :2]"""
          eye_landmarks = eye_landmarks[:, :2]
          eye_landmarks = np.asarray(eye_landmarks)
          eyelid_landmarks = eye_landmarks[0:8, :]
          iris_landmarks = eye_landmarks[8:16, :]
          print("iris centre ",eye_landmarks[16, :])
          iris_centre = eye_landmarks[16, :]
          eyeball_centre = eye_landmarks[17, :]
          eyeball_radius = np.linalg.norm(eye_landmarks[18, :] -
                                          eye_landmarks[17, :])
          
          i_x0, i_y0 = iris_centre
          e_x0, e_y0 = eyeball_centre
          theta = -np.arcsin(np.clip((i_y0 - e_y0) / eyeball_radius, -1.0, 1.0))
          phi = np.arcsin(np.clip((i_x0 - e_x0) / (eyeball_radius * -np.cos(theta)),
                                  -1.0, 1.0))
          current_gaze = np.array([theta, phi])
          print("current_gaze",current_gaze,current_gaze.shape)
          current_gaze = np.atleast_2d(current_gaze)
          predicted_gaze = np.atleast_2d(output['gaze'][1])
          print("predicted gaze",predicted_gaze, predicted_gaze.shape)
          error = angular_error(current_gaze,predicted_gaze)
          print("angular_error",error)
          mean_angular_errors.append(error)
          predicted.append(predicted_gaze)
          actual_gaze.append(current_gaze)
          print(i)
          i+=1

          


In [ ]:
testUnity_elg()

In [ ]:
import pandas as pd
predicted_df = pd.DataFrame(np.concatenate(predicted), columns = ['Yaw','Pitch'])
actual_df = pd.DataFrame(np.concatenate(actual_gaze), columns = ['Yaw','Pitch'])

In [ ]:
np.mean(mean_angular_errors)

In [ ]:
mean_angular_errors = [] 
predicted = []
actual_gaze = []

In [ ]:
predicted_df.head()
actual_df.head()

In [ ]:
import matplotlib.pyplot as plt

"""
plt.figure(figsize=(10,10))
plt.scatter(actual_df['Pitch'], predicted_df["Pitch"], c='crimson')
"""



fig, ax = plt.subplots()
ax.scatter(actual_df['Pitch'], predicted_df["Pitch"])
ax.plot([actual_df['Pitch'].min(), actual_df['Pitch'].max()], [actual_df['Pitch'].min(), actual_df['Pitch'].max()], 'k--', lw=4)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(actual_df['Yaw'], predicted_df["Yaw"])
ax.plot([actual_df['Yaw'].min(), actual_df['Yaw'].max()], [actual_df['Yaw'].min(), actual_df['Yaw'].max()], 'k--', lw=4)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()